## Note

Include notes here about notebook. Mention running "Build Datasets from Colab"

# Imports

In [4]:
run_imports=True
Google_Colab=True  # Running on Google Colab?

if run_imports:
    if Google_Colab==True:
        ## Mount Google Drive ##L
        from google.colab import drive
        drive.mount('/content/drive') # Mount Google Drive

        ## Iterstrat
        !pip install git+https://github.com/trent-b/iterative-stratification.git
        from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

    ## Python ##
    import os
    import time
    import sys
    from pathlib import Path
    import re
    #from IPython.display import display, clear_output

    ## Progress Bar
    !pip install tqdm
    from tqdm import tqdm

    ## Pytorch ##
    !pip install torch
    import torch
    from torch import nn
    from torch.utils.data import DataLoader
    from torch.utils.data import Dataset

    ## Torchvision ##
    !pip install torchvision
    from torchvision.utils import make_grid
    from torchvision import transforms

    ## Numpy/MatPlotLib ##
    !pip install numpy
    import numpy as np
    import re

    !pip install matplotlib
    import matplotlib.pyplot as plt
    from matplotlib.colors import Normalize
    from matplotlib.pyplot import savefig
    import matplotlib.gridspec as gridspec

    ## Pandas ##
    !pip install pandas
    import pandas as pd

    ## SciKit #
    !pip install scikit-image
    from skimage import metrics
    from skimage.metrics import structural_similarity
    from skimage.transform import radon, iradon
    from skimage.transform import iradon
    from skimage import morphology
    from skimage.morphology import opening, erosion
    #from skimage.restoration import denoise_bilateral, denoise_tv_chambolle, denoise_wavelet

    ## SciPy ##
    #from scipy.stats import moment as compute_moment

    ## UpRoot
    #import uproot

    ## Determine what Hardware we have ##
    device = ('cuda' if torch.cuda.is_available() else 'cpu')

    print('Device: ', device)

Mounted at /content/drive
  Cloning https://github.com/trent-b/iterative-stratification.git to /tmp/pip-req-build-a4pij46q
  Running command git clone --filter=blob:none --quiet https://github.com/trent-b/iterative-stratification.git /tmp/pip-req-build-a4pij46q
  Resolved https://github.com/trent-b/iterative-stratification.git to commit c763bd5440e5f03c6447471305897c1046fc4838
  Preparing metadata (setup.py) ... done
  Created wheel for iterative-stratification: filename=iterative_stratification-0.1.9-py3-none-any.whl size=8444 sha256=5f64993ada93659540e96db8373ec7331ba9d8508b4f56cafa2efccd48e3c851
  Stored in directory: /tmp/pip-ephem-wheel-cache-bchp0s8b/wheels/1a/53/b5/f5f9836cdfff718e36f6af0df976157168f38b5ce6bf53a49d
Successfully built iterative-stratification
Device:  cpu


# Misc. Functions

## Cropping

In [ ]:
def crop_single_image_by_size(image, crop_size=-1):
    '''
    Function to crop a single image to a square shape, with even margins around the edges.

    image:       Input image tensor of shape [height, width]
    crop_size:   Edge size of (square) image to keep. The edges are discarded.
    '''
    x_size = image.shape[1]

    margin_low = int((x_size-crop_size)/2.0)  # (90-71)/2 = 19/2 = 9.5 -->9
    margin_high = x_size-crop_size-margin_low # 90-71-9 = 10

    pix_min = 0 + margin_low
    pix_max = x_size - margin_high

    image = image[pix_min : pix_max , pix_min : pix_max]

    return image

def crop_single_image_by_factor(image, crop_factor=1):
    '''
    Function to crop a single image for a factor.

    image:       Input image tensor of shape [height, width]
    crop_factor: Fraction of image to keep. The image is trimmed so the edges are discarded.
    '''
    x_size = image.shape[1]
    y_size = image.shape[0]

    x_margin = int(x_size*(1-crop_factor)/2)
    y_margin = int(y_size*(1-crop_factor)/2)

    x_min = 0 + x_margin
    x_max = x_size - x_margin
    y_min = 0 + y_margin
    y_max = y_size - y_margin

    return image[y_min:y_max , x_min:x_max]


def crop_image_tensor_with_corner(image_tensor, crop_size, corner=(0,0)):
    '''
    Function which returns a smaller, cropped version of a tensor (multiple images)

    image_tensor:       a batch of images with dimensions: (num_images, channel, y_dimension, x_dimension)
    corner:      upper-left corner of window
    crop_size:   size of cropping window
    '''

    y_min = corner[0]
    y_max = corner[0]+crop_size
    x_min = corner[1]
    x_max = corner[1]+crop_size

    return image_tensor[:, :, y_min:y_max , x_min:x_max ]


def crop_image_tensor_by_size(image_tensor, crop_size):
    '''
    Function which returns a smaller, cropped version of a tensor (multiple images)

    batch:       a batch of images with dimensions: (num_images, channel, y_dimension, x_dimension)
    crop_size:   size of cropping window
    '''
    x_size = image_tensor.shape[3]
    y_size = image_tensor.shape[2]

    x_margin = int((x_size-crop_size)/2)
    y_margin = int((y_size-crop_size)/2)

    x_min = 0 + x_margin
    x_max = x_size - x_margin
    y_min = 0 + y_margin
    y_max = y_size - y_margin

    return image_tensor[:,:, y_min:y_max , x_min:x_max ]


def crop_image_tensor_by_factor(image_tensor, crop_factor=1):
    '''
    Function to crop an image tensor.

    image_tensor:   Input image tensor of shape [image number, channel, height, width]
    crop_factor:    Fraction of image to keep. The images are trimmed so the edges are discarded.
    '''
    x_size = image_tensor.shape[3]
    y_size = image_tensor.shape[2]

    x_margin = int(x_size*(1-crop_factor)/2)
    y_margin = int(y_size*(1-crop_factor)/2)

    x_min = 0 + x_margin
    x_max = x_size - x_margin
    y_min = 0 + y_margin
    y_max = y_size - y_margin

    return image_tensor[:,:, y_min:y_max , x_min:x_max ]

## Reconstructions & Projection

In [ ]:
def iradon_MLEM(sino_ground, azi_angles=None, max_iter=15, circle=True, crop_factor=2**0.5/2):
    '''
    Function to reconstruct a single PET image from a single sinogram using ML-EM.

    sino_ground:    sinogram (photopeak). This is a numpy array with minimum values of 0.
    azi_angles:     list of azimuthal angles for sinogram. If set to None, angles are assumed to span [0,180)
    max_iter:       Maximum number of iterations for ML-EM algorithm.
    circle:         circle=True: The projection data spans the width (or height) of the activity distribution, and the reconstructed image is circular.
                    circle=False: The projection data (sinograms) spans the corner-to-corner line of the activity distribution, and the reconstructed image is square.
    crop_size:      Size to crop the image to after performing ML-EM. For ML-EM performed on a 90x90 sinogram, the
                    output image will be 90x90. However, it is necessary to crop this to 64x64 to get the same FOV
                    as the dataset. This means the image must cropped by a factor of sqrt(2)/2.
    '''
    if azi_angles==None:
        num_angles = sino_ground.shape[1] # Width
        azi_angles=np.linspace(0, 180, num_angles, endpoint=False)

    ## Create Sensitivity Image ##
    sino_ones = np.ones(sino_ground.shape)
    sens_image = iradon(sino_ones, azi_angles, circle=circle, filter_name=None)

    if circle==False:
        def modify_sens(image, const_factor=0.9, slope=0.03):
            '''
            Modifies an image so that the area in the central FOV remains constant, but values at edges are attenuated.
            image               image to modify
            constant_factor     fraction of the image to leave alone
            slope               increase this to attenuate images at the edges more
            '''
            def shape_piecewise(r, const_value, slope):
                if r <= const_value:
                    return 1
                else:
                    return 1+slope*(r-const_value)

            y_max = image.shape[0]
            x_max = image.shape[1]

            const_dist = const_factor*x_max/2 # radius over which image remains constant

            x_center = (x_max-1)/2.0 # the -1 comes from the fact that the coordinates of a pixel start at 0, not 1
            y_center = (y_max-1)/2.0

            for y in range(0, y_max):
                for x in range(0, x_max):
                    r = ((x-x_center)**2 + (y-y_center)**2)**0.5

                    total_factor = shape_piecewise(r, const_dist, slope) # creates a circular shaped piece-wise
                    #total_factor = shape_piecewise(abs(x-x_center), const_dist, slope) * shape_piecewise(abs(y-y_center), const_dist, slope) # square-shaped piecewise
                    #total_factor = shape_piecewise(abs(y-y_center), const_dist, slope) # vertical only

                    image[y,x] = image[y,x]*total_factor

            return image
        sens_image = modify_sens(sens_image)

    ## Create blank reconstruction ##
    image_recon  = np.ones(sens_image.shape)

    for iter in range(max_iter):

        if circle==True:
            sens_image = sens_image + 0.001 # Guarantees the denominator is >0

        sino_recon = radon(image_recon, azi_angles, circle=circle) #
        sino_recon[sino_recon==0]=1000 # Set a limit on the denominator (next line)
        sino_ratio = sino_ground / (sino_recon) #
        image_ratio = iradon(sino_ratio, azi_angles, circle=circle, filter_name=None) / sens_image
        image_ratio[image_ratio>1.5]=1.5 # Sets limit on backprojected ratio, on how fast image can grow. Threshold and set value should equal each other (good value=1.5)
        image_recon = image_recon * image_ratio
        image_recon[image_recon<0]=0 # Sets floor on image pixels. No need to adjust.

        #footprint = morphology.disk(1)
        #image_recon = opening(image_recon, footprint)

    image_cropped = crop_single_image_by_factor(image_recon, crop_factor=crop_factor)
    #image_cropped = crop_single_image_by_size(image_recon, crop_size=crop_size)

    return image_cropped

def reconstruct(sinogram_tensor, config, image_size=90, recon_type='FBP', circle=True):
    '''
    Function for calculating a reconstructed PET image tensor, given a sinogram_tensor. One image is reconstructed for
    each sinogram in the sinogram_tensor.

    sinogram_tensor:    Tensor of sinograms of size (number of images)x(channels)x(height)x(width).
                        Only the first channel (photopeak) is used for recontruction here.
    config:             configuration dictionary
    image_size:         size of output (images are resized to this shape)
    recon_type:         Can be set to 'MLEM' for maximum-likelihood expectation maximization, or 'FBP' for
                        filtered back-projection.
    circle              circle=True: The projection data spans the width (or height) of the activity distribution, and the reconstructed image is circular.
                        circle=False: The projection data (sinograms) spans the corner-to-corner line of the activity distribution, and the reconstructed image is square.

    Function returns a tensor of reconstructed images. Returned images are resized, and optionall normalized and scaled (according to the keys in the configuration dictionary)
    '''
    normalize = config["SI_normalize"]
    scale = config['SI_scale']

    photopeak_array = torch.clamp(sinogram_tensor[:,0,:,:], min=0).detach().cpu().numpy()
    # Note: there really should be no need to clamp the sinogram as it should contain no negative values.

    ## Reconstruct Individual Sinograms ##
    first=True
    for sino in photopeak_array[0:,]:
        if recon_type == 'FBP':
            image = iradon(sino.squeeze(),
                        circle=False, # For an unknown reason, circle=False gives better reconstructions here. Maybe due to errors introduced in interpolation.
                        preserve_range=True,
                        filter_name='cosine' # Options: 'ramp', 'shepp-logan', 'cosine', 'hamming', 'hann'
                        )
        else:
            image = iradon_MLEM(sino, circle=circle)

        ## Morphologic Opening - removes outlier pixels than can cause problems with image normalization
        #footprint = morphology.disk(1)
        #image = opening(image, footprint)

        ## Concatenate Images ##
        image = np.expand_dims(image, axis=0) # Add a dimension to the beginning of the reconstructed image
        if first==True:
            image_array = image
            first=False
        else:
            image_array = np.append(image_array, image, axis=0)

    ## For All Images: create resized/dimensioned Torch tensor ##
    image_array = np.expand_dims(image_array, axis=1)        # Creates channels dimension
    a = torch.from_numpy(image_array)                        # Converts to Torch tensor
    a = torch.clamp(a, min=0)                                # You HAVE to clamp before normalizing or the negative values throw it off.
    a = transforms.Resize(size = (image_size, image_size), antialias=True)(a) # Resize tensor

    ## Normalize Entire Tensor ##
    if normalize:
        batch_size = len(a)
        a = torch.reshape(a,(batch_size, 1, image_size**2)) # Flattens each image
        a = nn.functional.normalize(a, p=1, dim = 2)
        a = torch.reshape(a,(batch_size, 1 , image_size, image_size)) # Reshapes images back into square matrices
        a = scale*a

    return a.to(device)

def project(image_tensor, circle=False, theta=-1):
    '''
    Perform the forward radon transform to calculate projections from images. Returns an array of sinograms.

    image_tensor:   tensor of PET images
    theta:          numpy array of projection angles. Default is [0,180)
    '''
    image_collapsed = torch.clamp(image_tensor[:,0,:,:], min=0).detach().squeeze().cpu().numpy()

    if theta==-1:
        theta = np.arange(0,180)

    first=True
    for image in image_collapsed[0:,]:
        sino = radon(image,
                    circle=circle,
                    preserve_range=True,
                    theta=theta,
                    )
        sino = np.moveaxis(np.atleast_3d(sino), 2, 0) # Adds a blank axis and moves it to the beginning
        if first==True:
            sino_array=sino
            first=False
        else:
            sino_array = np.append(sino_array, sino, axis=0)

    return torch.from_numpy(sino_array)

## Display Images

In [ ]:
def show_single_unmatched_tensor(image_tensor, grid=False, cmap='inferno', fig_size=1):
    '''
    Function for visualizing images. The images are displayed, each with their own colormap scaling, so quantitative comparisons are not possible.
    Send only the images you want plotted to this function. Works with both single channel and multi-channel images.
    If using the single-channel grid option, it plots 120 images in a 15x8 grid.

    image_tensor:   image tensor of shape [num, chan, height, width]
    grid:           If True, displays images in a 15x8 grid (120 images in total). If false, images are displayed in a horizontal line.
    cmap:           Matplotlib color map
    fig_size:       figure size
    '''
    print(f'Shape: {image_tensor.shape} // Min: {torch.min(image_tensor)} // Max: {torch.max(image_tensor)} \
    //Mean Sum (per image): {torch.sum(image_tensor).item()/(image_tensor.shape[0]*image_tensor.shape[1])} // Sum (a single image): {torch.sum(image_tensor[0,0,:])}')

    #image_tensor = image_tensor.detach().squeeze(.cpu()
    image_tensor = image_tensor.detach().cpu()
    image_tensor = torch.clamp(image_tensor, min=0)

    num = image_tensor.size(dim=0)
    chan = image_tensor.size(dim=1)

    ## Plot 3-Channel Images ##
    #image_np = image_grid.mean(dim=0).squeeze().numpy() # This also works!

    ## Plot Multi-Channel Images ##
    if chan!=1:
        #123
        print(f'Mean (Ch 0): {torch.mean(image_tensor[:,0,:,:])} // Mean (Ch 1): {torch.mean(image_tensor[:,1,:,:])} // Mean (Ch 2): {torch.mean(image_tensor[:,2,:,:])}')

        # Plot Grid #
        if grid:
            fig, ax = plt.subplots(num, chan, figsize=(fig_size*num, fig_size*chan), constrained_layout=True)
            for N in range(0, num): # Iterate over image number
                for C in range(0, chan): # Iterate over channels
                    img = image_tensor[N,C,:,:]
                    ax[N,C].axis('off')
                    ax[N,C].imshow(img.squeeze(), cmap=cmap)

        # Plot in-Line #
        else:
            fig, ax = plt.subplots(1, num*(chan+1), figsize=(fig_size, fig_size*num*(chan+1)), constrained_layout=True)
            i=0
            for N in range(0, num): # Iterate over image number
                for C in range(0, chan): # Iterate over channels
                    img = image_tensor[N,C,:,:]
                    ax[i].axis('off')
                    ax[i].imshow(img.squeeze(), cmap=cmap)
                    i+=1
                blank = torch.ones_like(img)
                ax[i].axis('off')
                ax[i].imshow(blank.squeeze())
                i+=1

    ## Plot 1 Channel Images ##
    else:
        # Plot Grid #
        # Note: This plots 120 images at a time!
        if grid:
            cols, rows = 15, 8
        # Plot in-Line #
        else:
            rows = 1
            cols = image_tensor.shape[0]

        figure=plt.figure(figsize=(cols*fig_size,rows*fig_size))

        for i in range(0, cols*rows):
            img = image_tensor[i]             # Shape: torch.Size([3, 1, 180, 180]) /
            figure.add_subplot(rows,cols,i+1) # MatplotLib indeces start at 1
            plt.axis("off")
            plt.imshow(img.squeeze(), cmap=cmap)

    plt.show()


def show_multiple_matched_tensors(*image_tensors, cmap='inferno', fig_size=0.8):
    '''
    Function for visualizing images from multiple tensors. Each image is "matched" with images from the other tensors,
    and each matched set of images (one from each tensor) is plotted with the same colormap in a column.
    Send only the images you want plotted to this function. Works with both single channel and multi-channel images.

    image_tensors:  list of tensors, each of which may contain multiple images.
    '''
    for tensor in image_tensors:
        # Begin by printing statistics for each tensor
        print(f'Shape: {tensor.shape} // Min: {torch.min(tensor)} // Max: {torch.max(tensor)} \
        // Mean: {torch.mean(tensor)} // Mean Sum (per image): {torch.sum(tensor).item()/(tensor.shape[0]*tensor.shape[1])} // Sum (a single image): {torch.sum(tensor[0,0,:])}')

    combined_tensor = torch.cat(image_tensors, dim=0).detach().cpu()
    combined_tensor = torch.clamp(combined_tensor, min=0)

    num_rows = len(image_tensors)           # The number of rows equals the number of tensors (images to match)
    num_cols = len(image_tensors[0])        # The length of the zeroth element (of the list) is the number of images in a tensor.
    num_chan = image_tensors[0].size(dim=1) # Equivalent to: image_tensors[0].shape(1)

    ## Plot 1 Channel Images ##
    if num_chan==1:
        fig, ax = plt.subplots(num_rows, num_cols, squeeze=False, figsize=(fig_size*num_cols, fig_size*num_rows), constrained_layout=True)
        #fig, ax = plt.subplots(num_rows, num_cols, constrained_layout=True)

        i=0 # i = column number
        for col in range(0, num_cols): # Iterate over column number. All images in a column will have the same colormap.
            img_list=[]
            min_list=[]
            max_list=[]

            # Construct image list and normalization object for matched images in a column (iterating over rows) #
            for row in range(0,num_rows):                               # We iterate over rows in orcer
                img = combined_tensor[row*num_cols+col, 0 ,:,:]         # Grab the correct image (zeroth channel for 1-D images)
                img_list.append(img)                                    # We construct a new image list for each row
                min_list.append(torch.min(img).item())                  # Create list of image minimums
                max_list.append(torch.max(img).item())                  # Create list of image maximums
            norm = Normalize(vmin=min(min_list), vmax=max(max_list))    # We construct a normalization object with min/max = min/max pixel value for all images in list

            # Plot normalized images in a single column (iterating over rows) #
            for row in range(0,num_rows):
                ax[row, i].axis('off')
                ax[row, i].imshow(img_list[row].squeeze(), cmap=cmap, norm=norm) # Squeeze gets rid of extra channel dimension
            i+=1

    ## Plot Multi-Channel Images ##
    else:
        print(f'Mean (Ch 0): {torch.mean(combined_tensor[:,0,:,:])} // Mean (Ch 1): {torch.mean(combined_tensor[:,1,:,:])} // Mean (Ch 2): {torch.mean(combined_tensor[:,2,:,:])}')

        #if num_cols>3:  # Restricts to 3-channels. You could get rid of this without an issue.
        #    num_cols=3

        # Construct figure and axes. Note: 'num_chan+1' arises from the divider blank image btw. each multi-channel image
        fig, ax = plt.subplots(num_rows, num_cols*(num_chan+1), squeeze=False, figsize=(fig_size*num_cols*(num_chan+1), fig_size*num_rows), constrained_layout=True)

        i=0
        for col in range(0, num_cols):      # Iterate over column number
            for chan in range(0, num_chan): # Iterate over channels
                img_list=[]
                min_list=[]
                max_list=[]

                # Iterates over rows (one row per tensor) to construct an image list and normalization object a single column. All matched images have the same channel. #
                for row in range(0,num_rows):
                    img = combined_tensor[row*num_cols+col, chan ,:,:] # Constructs an image list where each row has the same channel #
                    img_list.append(img)
                    min_list.append(torch.min(img).item())
                    max_list.append(torch.max(img).item())
                norm = Normalize(vmin=min(min_list), vmax=max(max_list))

                # Iterates over rows to plot matched images in a single column. These share the same channel. #
                for row in range(0,num_rows):
                    ax[row, i].axis('off')
                    ax[row, i].imshow(img_list[row].squeeze(), cmap=cmap, norm=norm) # Squeeze gets rid of extra channel dimension
                i+=1

            # After all channels have been iterated, the complete multi-channel image has been plotted. Now we plot a divider before the next image #
            for row in range(0,num_rows):
                blank = torch.ones_like(img)
                ax[row, i].axis('off')
                ax[row, i].imshow(blank.squeeze())
            i+=1

    plt.show()

def show_single_commonmap_tensor(image_tensor, nrow=15, figsize=(27,18), cmap='inferno'):
    '''
    Function for visualizing images from one tensor, all of which will be plotted with the same scaled colormap. Only works with single-channel image tensors.

    image_tensor:  image tensor. nrow should go into this evenly.
    nrow:          number of rows for the image grid
    figsize:       figure size
    cmap:          color map
    '''
    tensor = torch.clamp(image_tensor, min=0).detach().cpu()
    image_grid = make_grid(tensor, nrow=nrow)  # from torchvision.utils import make_grid

    #print(f'Shape: {tensor.shape} // Min: {torch.min(tensor)} // Max: {torch.max(tensor)} \
    #// Mean: {torch.mean(tensor)} // Mean Sum (per image): {torch.sum(tensor).item()/(tensor.shape[0]*tensor.shape[1])} // Sum (a single image): {torch.sum(tensor[0,0,:])}')

    fig, ax = plt.subplots(1,1, figsize=figsize)
    ax.axis('off')

    image_grid = image_grid[0,:].squeeze()
    #plt.imshow(image_grid, cmap=cmap)
    im = ax.imshow(image_grid, cmap=cmap)
    #fig.colorbar(im, ax=ax)
    plt.show()

def show_multiple_commonmap_tensors(*image_tensors, cmap='inferno'):
    '''
    Function for visualizing images from multiple tensors, all of which will be plotted with the same scaled colormap. Only works with single-channel image tensors.

    *image_tensors: list of image tensors, all of which should contain the same number of images. Only send the number of images you want to plot to this function.
    '''
    # Print tensor statistics #
    for tensor in image_tensors:
        print(f'Shape: {tensor.shape} // Min: {torch.min(tensor)} // Max: {torch.max(tensor)} \
        // Mean: {torch.mean(tensor)} // Mean Sum (per image): {torch.sum(tensor).item()/(tensor.shape[0]*tensor.shape[1])} // Sum (a single image): {torch.sum(tensor[0,0,:])}')

    num_rows = len(image_tensors)
    num_columns = len(image_tensors[0])
    # Combine tensors into one & clamp #
    combined_tensor = torch.cat(image_tensors, dim=0).detach().cpu()
    combined_tensor = torch.clamp(combined_tensor, min=0)
    # Make a grid of the tensors #
    image_grid = make_grid(combined_tensor, nrow=num_columns) # Note: nrow is the number of images displayed in each row (i.e., the number of columns)

    # Determine figure size #
    print('num_rows:', num_rows)
    fig, ax = plt.subplots(1,1, figsize=(30,1*num_rows))
    #fig, ax = plt.subplots(1,1, figsize=(30,7))

    ax.axis('off')

    image_grid = image_grid[0,:].squeeze()
    im = ax.imshow(image_grid, cmap=cmap)
    fig.colorbar(im, ax=ax)
    plt.show()

## Metrics

In [ ]:
##################################################
## Functions for Calculating Metrics Dataframes ##
##################################################

## Calculate Arbitrary Metric ##
def calculate_metric(batch_A, batch_B, img_metric_function, return_dataframe=False, label='default', crop_factor=1):
    '''
    Function which calculates metric values for two batches of images.
    Returns either the average metric value for the batch or a dataframe of individual image metric values.

    batch_A:                tensor of images to compare [num, chan, height, width]
    batch_B:                tensor of images to compare [num, chan, height, width]
    img_metric_function:    a function which calculates a metric (MSE, SSIM, etc.) from two INDIVIDUAL images
    return_dataframe:       If False, then the average is returned.
                            Otherwise both the average, and a dataframe containing the metric values of the images in the batches, are returned.
    label:                  what to call dataframe, if it is created
    crop_factor:            factor by which to crop both batches of images. 1 = whole image retained.
    '''

    if crop_factor != 1:
        A = crop_image_tensor_by_factor(batch_A, crop_factor=crop_factor)
        B = crop_image_tensor_by_factor(batch_B, crop_factor=crop_factor)

    length = len(batch_A)
    metric_avg = 0
    metric_list = []

    for i in range(length):
        image_A = batch_A[i:i+1,:,:,:] # Using i:i+1 instead of just i preserves the dimensionality of the array
        image_B = batch_B[i:i+1,:,:,:]

        metric_value = img_metric_function(image_A, image_B)
        metric_avg += metric_value/length
        if return_dataframe==True:
            metric_list.append(metric_value)

    metric_frame = pd.DataFrame({label : metric_list})

    if return_dataframe==False:
        return metric_avg
    else:
        return metric_frame, metric_avg


def update_tune_dataframe(tune_dataframe, model, config, mean_CNN_MSE, mean_CNN_SSIM, mean_CNN_CUSTOM):
    '''
    Function to update the tune_dataframe for each trial run that makes it partway through the tuning process.
    '''
    # Extract values from config dictionary
    SI_dropout =        config['SI_dropout']
    SI_exp_kernel =     config['SI_exp_kernel']
    SI_gen_fill =       config['SI_gen_fill']
    SI_gen_hidden_dim = config['SI_gen_hidden_dim']
    SI_gen_neck =       config['SI_gen_neck']
    SI_layer_norm =     config['SI_layer_norm']
    SI_normalize =      config['SI_normalize']
    SI_pad_mode =       config['SI_pad_mode']
    batch_size =        config['batch_size']
    gen_lr =            config['gen_lr']

    # Calculate number of trainable weights in CNN
    num_params = sum(map(torch.numel, model.parameters()))

    # Concatenate Dataframe
    add_frame = pd.DataFrame({'SI_dropout': SI_dropout, 'SI_exp_kernel': SI_exp_kernel, 'SI_gen_fill': SI_gen_fill, 'SI_gen_hidden_dim': SI_gen_hidden_dim,
                            'SI_gen_neck': SI_gen_neck, 'SI_layer_norm': SI_layer_norm, 'SI_normalize': SI_normalize, 'SI_pad_mode': SI_pad_mode, 'batch_size': batch_size,
                            'gen_lr': gen_lr, 'num_params': num_params, 'mean_CNN_MSE': mean_CNN_MSE, 'mean_CNN_SSIM': mean_CNN_SSIM, 'mean_CNN_CUSTOM': mean_CNN_CUSTOM}, index=[0])

    tune_dataframe = pd.concat([tune_dataframe, add_frame], axis=0)

    # Save Dataframe to File
    tune_dataframe.to_csv(tune_dataframe_path, index=False)

    return tune_dataframe


def update_test_dataframe_and_return_outputs(sino_tensor, image_size, CNN_output, ground_image, test_dataframe, config):
    '''
    Function which: A) performs reconstructions (FBP and possibly ML-EM)
                    B) constructs a dataframe of metric values (MSE & SSIM) for these reconstructions, and also for the CNN output, with respect to the ground truth image.
                    C) concatenates this with the test dataframe passed to this function
                    D) returns the concatenated dataframe, mean metric values, and reconstructions

    sino_tensor:    sinogram tensor of shape [num, chan, height, width]
    image_size:     image_size
    CNN_output:     CNN reconstruction
    ground_image:   ground truth image
    test_dataframe: dataframe to append metric values to
    config:         general config dictionary
    '''
    # Construct Outputs #
    FBP_output = reconstruct(sino_tensor, config, image_size=image_size, recon_type='FBP')
    if compute_MLEM==True:
        MLEM_output = reconstruct(sino_tensor, config, image_size=image_size, recon_type='MLEM')
    else: # If not looking at ML-EM, don't waste time computing the MLEM images, which can take awhile.
        MLEM_output = FBP_output

    # Dataframes: build dataframes for every reconstruction technique/metric combination #
    batch_CNN_MSE,  mean_CNN_MSE   = calculate_metric(ground_image, CNN_output, MSE,  return_dataframe=True, label='MSE (Network)')
    batch_CNN_SSIM,  mean_CNN_SSIM = calculate_metric(ground_image, CNN_output, SSIM, return_dataframe=True, label='SSIM (Network)')
    batch_FBP_MSE,  mean_FBP_MSE   = calculate_metric(ground_image, FBP_output, MSE,  return_dataframe=True, label='MSE (FBP)')
    batch_FBP_SSIM,  mean_FBP_SSIM = calculate_metric(ground_image, FBP_output, SSIM, return_dataframe=True, label='SSIM (FBP)')
    batch_MLEM_MSE, mean_MLEM_MSE  = calculate_metric(ground_image, MLEM_output, MSE, return_dataframe=True, label='MSE (ML-EM)')
    batch_MLEM_SSIM, mean_MLEM_SSIM= calculate_metric(ground_image, MLEM_output, SSIM,return_dataframe=True, label='SSIM (ML-EM)')

    # Concatenate batch dataframes and larger running test dataframe
    add_frame = pd.concat([batch_CNN_MSE, batch_FBP_MSE, batch_MLEM_MSE, batch_CNN_SSIM, batch_FBP_SSIM, batch_MLEM_SSIM], axis=1)
    test_dataframe = pd.concat([test_dataframe, add_frame], axis=0)

    # Return a whole lot of stuff
    return test_dataframe, mean_CNN_MSE, mean_CNN_SSIM, mean_FBP_MSE, mean_FBP_SSIM, mean_MLEM_MSE, mean_MLEM_SSIM, FBP_output, MLEM_output

######################
## Metric Functions ##
######################

## Metrics which take only single images as inputs ##
## ----------------------------------------------- ##
def SSIM(image_A, image_B, win_size=-1):
    '''
    Function to return the SSIM for two 2D images.

    image_A:        pytorch tensor for a single image
    image_B:        pytorch tensor for a single image
    win_size:       window size to use when computing the SSIM. If =-1, the full size of the image is used.
    '''

    if win_size == -1:   # The default shape of the window size is the same size as the image.
        x = image_A.shape[2]
        win_size = (x if x % 2 == 1 else x-1) # Guarantees the window size is odd.

    image_A_npy = image_A.detach().squeeze().cpu().numpy()
    image_B_npy = image_B.detach().squeeze().cpu().numpy()

    max_value = max([np.amax(image_A_npy, axis=(0,1)), np.amax(image_B_npy, axis=(0,1))])   # Find maximum among the images
    min_value = min([np.amin(image_A_npy, axis=(0,1)), np.amin(image_B_npy, axis=(0,1))])   # Find minimum among the images
    data_range = max_value-min_value

    SSIM_image = structural_similarity(image_A_npy, image_B_npy, data_range=data_range, gaussian_weights=False, use_sample_covariance=False, win_size=win_size)

    return SSIM_image

## Metrics which take either batches or images as inputs ##
## ----------------------------------------------------- ##
def MSE(image_A, image_B):
    '''
    Function to return the mean square error for two 2D images (or two batches of images).

    image_A:        pytorch tensor for a single image
    image_B:        pytorch tensor for a single image
    '''
    image_A_npy = image_A.detach().squeeze().cpu().numpy()
    image_B_npy = image_B.detach().squeeze().cpu().numpy()

    return torch.mean((image_A-image_B)**2).item()

def NMSE(image_A, image_B):
    '''
    Function to return the normalized mean square error for two 2D images (or two batches of images).

    image_A:        pytorch tensor for a single image (reference image)
    image_B:        pytorch tensor for a single image
    '''
    image_A_npy = image_A.detach().squeeze().cpu().numpy()
    image_B_npy = image_B.detach().squeeze().cpu().numpy()

    return (torch.mean((image_A-image_B)**2)/torch.mean(image_A**2)).item()

def MAE(image_A, image_B):
    '''
    Function to return the mean absolute error for two 2D images (or two batches of images).

    image_A:        pytorch tensor for a single image
    image_B:        pytorch tensor for a single image
    '''
    image_A_npy = image_A.detach().squeeze().cpu().numpy()
    image_B_npy = image_B.detach().squeeze().cpu().numpy()

    return torch.mean(torch.abs(image_A-image_B)).item()

def calculate_moments(batch_A, batch_B, window_size = 10, stride=10, dataframe=False):
    '''
    Function to return the three statistical moment scores for two image tensors.
    '''
    ## Nested Functions ##

    def compare_moments(win_A, win_B, moment):
        def compute_moment(win, moment, axis=1):
            mean_value = np.mean(win, axis=axis)
            if moment == 1:
                return mean_value
            else:
                mean_array = np.array([mean_value] * win.shape[1]).T  # The square brackets in win.shape[1] mean the value is repeated spatially
                moment = np.mean((win - mean_array)**moment, axis=1)
                return moment

        batch_size = win_A.shape[0]


        reshape_A = (torch.reshape(win_A, (batch_size, -1))).detach().cpu().numpy()
        reshape_B = (torch.reshape(win_B, (batch_size, -1))).detach().cpu().numpy()

        moment_A = compute_moment(reshape_A, moment=moment)
        moment_B = compute_moment(reshape_B, moment=moment)
        moment_score = np.mean(np.absolute(moment_A-moment_B)/(np.absolute(moment_A)+0.1))

        '''
        print('===============================')
        print('MOMENT: ', moment)
        print('moment_A shape: ', moment_A.shape)
        print('moment_A mean: ', np.mean(moment_A))
        print('moment_B shape: ', moment_B.shape)
        print('moment_B mean: ', np.mean(moment_B))
        print('moment_score, |moment_A-moment_B|/(moment_A+0.1) : ', moment_score)
        '''
        return moment_score

    ## Code ##
    image_size = batch_A.shape[2]

    num_windows = int((image_size)/stride) # Maximum number of windows occurs when: stride = window_size.
    while (num_windows-1)*stride + window_size > image_size: # Solve for the number of windows (crops)
        num_windows += -1

    moment_1_running_score = 0
    moment_2_running_score = 0
    moment_3_running_score = 0

    for i in range(0, num_windows):
        for j in range(0, num_windows):
            corner = (i*stride, j*stride)

            win_A = crop_image_tensor_with_corner(batch_A, window_size, corner)
            win_B = crop_image_tensor_with_corner(batch_B, window_size, corner)

            moment_1_score = compare_moments(win_A, win_B, moment=1)
            moment_2_score = compare_moments(win_A, win_B, moment=2)
            moment_3_score = compare_moments(win_A, win_B, moment=3)

            moment_1_running_score += moment_1_score
            moment_2_running_score += moment_2_score
            moment_3_running_score += moment_3_score

    return moment_1_running_score, moment_2_running_score, moment_3_running_score

def LDM(batch_A, batch_B):
    '''
    Calculate the local distributions metric (LDM) for two batches of images
    '''

    score_1, score_2, score_3 = calculate_moments(batch_A, batch_B, window_size=5, stride=5)

    score_1 = score_1*1
    score_2 = score_2*1
    score_3 = score_3*1

    '''
    print('Scores')
    print('====================')
    print(score_1)
    print(score_2)
    print(score_3)
    '''

    return score_1+score_2+score_3

def custom_metric(batch_A, batch_B):
    return 0
    #return MSE(batch_A, batch_B)



###############################################
## Average or a Batch Metrics: Good for GANs ##
###############################################

# Range #
def range_metric(real, fake):
    '''
    Computes a simple metric which penalizes "fake" images in a batch for having a range different than the "real" images in a batch.
    Only a single metric number is returned.
    '''
    range_real = torch.max(real).item()-torch.min(real).item()
    range_fake = torch.max(fake).item()-torch.min(fake).item()

    return abs(range_real-range_fake)/(range_real+.1)

# Average #
def avg_metric(real, fake):
    '''
    Computes a simple metric which penalizes "fake" images in a batch for having an average value different than the "real" images in a batch.
    Only a single metric number is returned.
    '''
    avg_metric = abs((torch.mean(real).item()-torch.mean(fake).item())/(torch.mean(real)+.1).item())
    return avg_metric

# Pixel Variation #
def pixel_dist_metric(real, fake):
    '''
    Computes a metric which penalizes "fake" images for having a pixel distance different than the "real" images.

    real: real image tensor
    fake: fake image tensor
    '''
    def pixel_dist(image_tensor):
        '''
        Function for computing the pixel distance (standard deviation from mean) for a batch of images.
        For simplicity, it only looks at the 0th channel.
        '''
        array = image_tensor[:,0,:,:].detach().cpu().numpy().squeeze()
        sd = np.std(array, axis=0)
        avg=np.mean(sd)
        return(avg)

    pix_dist_fake = pixel_dist(fake)
    pix_dist_real = pixel_dist(real)

    return abs((pix_dist_real-pix_dist_fake)/(pix_dist_real+.1)) # The +0.1 in the denominators guarantees we don't divide by zero

###################
## Old Functions ##
###################


def LDM_OLD(real, fake, crop_size = 10, stride=10):
    '''
    Function to return the local distributions metric for two images.

    image_A:        pytorch tensor for a single image
    image_B:        pytorch tensor for a single image
    '''
    image_size = real.shape[2]

    i_max = int((image_size)/stride) # Maximum number of windows occurs when the stride equals the crop_size
    while (i_max-1)*stride + crop_size > image_size: # If stride < crop_size, we need fewer need to solve for the number of crops
        i_max += -1

    def crop_image_tensor_with_corner(A, corner=(0,0), crop_size=1):
        '''
        Function which returns a small, cropped version of an image.

        A           a batch of images with dimensions: (num_images, channel, height, width)
        corner      upper-left corner of window
        crop_size   size of croppiong window
        '''
        x_min = corner[1]
        x_max = corner[1]+crop_size
        y_min = corner[0]
        y_max = corner[0]+crop_size
        return A[:,:, y_min:y_max , x_min:x_max ]

    running_dist_score = 0
    running_avg_score = 0

    for i in range(0, i_max):
        for j in range(0, j_max):
            corner = (i*crop_size, j*crop_size)
            win_real = crop_image_tensor_with_corner(real, corner, crop_size)
            win_fake = crop_image_tensor_with_corner(fake, corner, crop_size)

            #range_score = range_metric(win_real, win_fake)
            avg_score = avg_metric(win_real, win_fake)
            pixel_dist_score = pixel_dist_metric(win_real, win_fake)

            running_dist_score += pixel_dist_score
            running_avg_score += avg_score

    combined_score = running_dist_score + running_avg_score

    return combined_score


## ROI Analysiss

In [ ]:
def ROI_NEMA_hot(ground_truth_tensor, reconstruction_tensor, background_mask, hot_mask):

    background_truth =          np.sum(np.multiply(background_mask.numpy(), ground_truth_tensor.numpy())) # You must convert tensors to numpy arrays before multiplication
    background_reconstruction = np.sum(np.multiply(background_mask.numpy(), reconstruction_tensor.numpy()))

    hot_truth          = np.sum(np.multiply(hot_mask.numpy(), ground_truth_tensor.numpy()))
    hot_reconstruction = np.sum(np.multiply(hot_mask.numpy(), reconstruction_tensor.numpy()))

    print('background_reconstruction: ', background_reconstruction)
    print('background_truth: ', background_truth)
    print('hot_reconstruction: ', hot_reconstruction)
    print('hot_truth: ', hot_truth)

    x = 100*(hot_reconstruction/background_reconstruction-1)/(hot_truth/background_truth-1) # This measure is invariant to absolute pixel values. It only measure relative contrast.

    return x


def ROI_NEMA_cold(reconstruction_tensor, background_mask, cold_mask):

    background_reconstruction = np.sum(np.multiply(background_mask.numpy(), reconstruction_tensor.numpy()))
    cold_reconstruction = np.sum(np.multiply(cold_mask.numpy(), reconstruction_tensor.numpy()))

    print('background_reconstruction: ', background_reconstruction)
    print('cold_reconstruction: ', cold_reconstruction)
    x = 100*(1-cold_reconstruction/background_reconstruction) # This measure does not take into account ground truths because the cold/background should ideally always be zero.

    return x

# Find Train/Val/Test Splits

In [ ]:
def stratified_split_from_csv(csv_path, stratify_cols, code_col,
                              proportions=(0.6, 0.2, 0.2),
                              output_txt="phantom_splits.txt",
                              random_state=42):
    """
    Splits a phantom dataset into train/validation/test sets using stratified sampling.

    Parameters
    ----------
    csv_path : str
        Path to the input CSV file containing phantom characteristics.
    stratify_cols : list of str
        Column names to use for stratified splitting (e.g. ["age", "gender", "BMI"]).
    code_col : str
        Column name that uniquely identifies each phantom.
    proportions : tuple of floats
        Fractions for (train, val, test) sets. Must sum to 1.
    output_txt : str
        Path for the output text file listing phantom codes per set.
    random_state : int
        Seed for reproducibility.

    Returns
    -------
    train_codes, val_codes, test_codes : list of str
        Lists of phantom codes for each split.
    """
    # --- 1. Read CSV ---
    df = pd.read_csv(csv_path)

    # --- 2. Automatically bin numeric columns for stratification ---
    df_binned = df.copy()
    for col in stratify_cols:
        if np.issubdtype(df[col].dtype, np.number):
            # Automatically bin into 4 roughly equal-frequency bins. Column entries are replace with strings like '(19.0, 32.0]'. These labels repeat for numbers in the same range.
            df_binned[col] = pd.qcut(df[col], q=4, duplicates='drop').astype(str)
        else:
            df_binned[col] = df[col].astype(str) # Makes sure all labels are strings.

    # --- 3. Convert stratification columns into a multilabel one-hot matrix ---
    strat_matrix = pd.get_dummies(df_binned[stratify_cols]) # strat_matrix is still a datafram.

    # --- 4. Prepare for stratified splitting ---
    X = np.zeros((len(df), 1))  # dummy feature matrix (not used)
    y = strat_matrix.values     # y = multilabel targets for stratification. The .values attribute means that y is a numpy matrix now, not a dataframe.

    msss = MultilabelStratifiedShuffleSplit(n_splits=1,  # Create an msss object with method that splits phantoms into (training) set and [(validation + test) set].
                                            test_size=proportions[1] + proportions[2],
                                            random_state=random_state)
    train_idx, temp_idx = next(msss.split(X, y)) # These are the absolute indices identifying the (training set) and [(validation + test) set]

    # Split remaining into validation and test
    temp_y = y[temp_idx]  # temp_y = matrix of one hot vectors for remaining sets (validation + test)
    test_fraction = proportions[2] / (proportions[1] + proportions[2])  # Proportion of remaining (validation + test) phantoms that are composed of validation phantoms
    msss_val = MultilabelStratifiedShuffleSplit(n_splits=1,            # We set n_splits=1 because if we try doing multiple splits at once, we'll just get multiple instances of partitioning the set in two.
                                                test_size=test_fraction,
                                                random_state=random_state)
    val_idx_rel, test_idx_rel = next(msss_val.split(np.zeros((len(temp_y), 1)), temp_y))

    val_idx = temp_idx[val_idx_rel]  # Converts from relative index back to absolute
    test_idx = temp_idx[test_idx_rel]

    # --- 5. Extract phantom codes ---
    codes = df[code_col].astype(str)
    train_codes = list(codes.iloc[train_idx])
    val_codes = list(codes.iloc[val_idx])
    test_codes = list(codes.iloc[test_idx])

    # --- 6. Write results to a text file ---
    with open(output_txt, "w") as f:
        f.write("TRAIN SET:\n")
        f.write(", ".join(train_codes) + "\n\n")
        f.write("VALIDATION SET:\n")
        f.write(", ".join(val_codes) + "\n\n")
        f.write("TEST SET:\n")
        f.write(", ".join(test_codes) + "\n")

    return train_codes, val_codes, test_codes


## 52 Patients total ##
# 35 train
# 10 test
# 7  cross-validation

csv_path = '/content/drive/MyDrive/Colab/Working/dataset/XCAT_sortingInfo.csv'
stratify_cols = ['Gender', 'Age', 'BMI']
code_col = 'Code'
proportions = (35/52.0,7/52,10/52)
output_txt = '/content/drive/MyDrive/Colab/Working/dataset/phantom_splits.txt'
random_state=33

train_codes, val_codes, test_codes = stratified_split_from_csv(csv_path, stratify_cols, code_col, proportions=proportions, output_txt=output_txt, random_state=random_state)

print('Train Codes: ', train_codes)
print('Validation Codes ', val_codes)
print('Test Codes: ', test_codes)

print(len(train_codes))
print(len(val_codes))
print(len(test_codes))
print(len(train_codes)+len(val_codes)+len(test_codes))


Train Codes:  ['male_pt96', 'male_pt108', 'male_pt118', 'male_pt139', 'male_pt141', 'male_pt144', 'male_pt145', 'male_pt155', 'male_pt157', 'male_pt163', 'male_pt164', 'male_pt167', 'male_pt169', 'male_pt171', 'male_pt178', 'male_pt180', 'male_pt184', 'male_pt196', 'male_pt200', 'male_pt168', 'female_pt71', 'female_pt76', 'female_pt86', 'female_pt89', 'female_pt98', 'female_pt99', 'female_pt117', 'female_pt152', 'female_pt153', 'female_pt166', 'female_pt175', 'female_pt176', 'female_pt182', 'female_pt170']
Validation Codes  ['male_pt93', 'male_pt128', 'male_pt159', 'male_pt201', 'female_pt106', 'female_pt147', 'female_pt149']
Test Codes:  ['male_pt77', 'male_pt80', 'male_pt92', 'male_pt150', 'male_pt154', 'male_pt173', 'female_pt140', 'female_pt142', 'female_pt143', 'female_pt151', 'female_pt162']
34
7
11
52


# Create Dataset from Interfiles

## Read Interfile

In [ ]:
def read_interfile_data(load_dir_path, header_filename, exclude_slices=(0,0), numpy=False, shift_x=0, shift_y=0, crop_size=-1, resize_size=-1, blur=-1, normalize_scale=-1):
    '''
    Function for reading interfile images or sinograms. Either may be in interfile 3.3 or STIR format.
        The function will figure which of the four possibilities (2 formats x 2 file types) are being read.
        Returns a pytorch tensor.

    load_dir_path:   path to directory containing the data
    header_file:     header file (of interfile). Can use any extension.
    exclude_slices:  tuple. Exclude this number of slices at the beginning and end of stack, keeping only the middle slices.
    numpy:           return a numpy array instead of a pytorch tensor.
    shift_x:         translate the image this many pixels to the right
    shift_y:         translate the image this many pixels up


    ## The following parameters may be set to -1 to disable cropping, resizing, blurring, or normalization.

    crop_size:       crop the square tensor to be of shape: [ #images, 1, crop_size, crop_size]
    resize_size:     set to a positive scalar <=1 (resize by a scale factor), or tuple (resize to a specific size)
    blur:            set to a tuple for gaussian blur (kernel size, s.d.)
    normalize_scale: normalize pixel values so that they add to this parameter

    '''

    def read_interfile_header(header_path):
        """Read and parse the Interfile header."""
        header_info = {}
        with open(header_path, 'r') as file:
            for line in file:
                #if line.startswith('!'):
                #    continue
                key_value = re.split(r':=', line, maxsplit=1)
                if len(key_value) == 2:
                    key = key_value[0].strip()
                    value = key_value[1].strip()
                    header_info[key] = value
        return header_info


    header_path=os.path.join(load_dir_path, header_filename)
    header_info=read_interfile_header(header_path)

    ## Determine file type ##
    if int(header_info['!matrix size [1]'])>380: # Sinograms have first dimension = 381
        if os.path.splitext(header_path)[1]=='.hs':
            type='sino_hs'  # Sinogram in interfile 3.33 format
        else:
            type='sino_v'   # Sinogram in STIR format
    else:
        type='image'

    ## If Image ##
    if type=='image':
        x_size = int(header_info['!matrix size [1]'])
        y_size = int(header_info['!matrix size [2]'])
        if '!matrix size [3]' in header_info:
            axial_size = int(header_info['!matrix size [3]'])
        else:
            axial_size = int(header_info['!total number of images'])
        matrix_size = np.array([axial_size,y_size,x_size])

    ## If Sinogram ##
    else:
        tangential_size = int(header_info['!matrix size [1]'])
        if type=='sino_hs':
            view_size = int(header_info['!matrix size [3]'])    # view_size = number of projections
            axial_size_str = header_info['!matrix size [2]']    # In this case, the axial coordinate is surrounded by {}
            match = re.search(r'\d+', axial_size_str)
            axial_size = int(match.group())
        elif type=='sino_v':
            view_size = int(header_info['!matrix size [2]'])
            axial_size = int(header_info['!matrix size [3]'])
        matrix_size = np.array([axial_size, view_size, tangential_size])     # Here, the angles are in the vertical dimension. That is altered below.

    ## Read the binary data ##
    if 'name of data file' in header_info:
        bin_filename = header_info['name of data file']
    else:
        bin_filename = header_info['!name of data file']
    bin_path=os.path.join(load_dir_path, bin_filename)
    total_elements = np.prod(matrix_size)
    bin_array = np.fromfile(bin_path, dtype=np.float32, count=total_elements)

    ## Manipulate & Convert 3D Stack ##
    bin_array = bin_array.reshape(matrix_size)
    bin_array = np.expand_dims(bin_array, axis=1)  # Creates channels dimension
    bin_tensor = torch.from_numpy(bin_array)       # Converts to PyTorch tensor
    bin_tensor = torch.clamp(bin_tensor, min=0)    # You HAVE to clamp before normalizing or the negative values throw it off.

    ## If Sinogram, rotate & flip tensor ##
    if type=='sino_hs' or type=='sino_v':
        bin_tensor = transforms.functional.rotate(bin_tensor, 90, fill=0, expand=True) # Rotate image (now the angles are on the horizontal dimension)
        bin_tensor = torch.flip(bin_tensor, dims=(3,)) # Flip horizontally
        #bin_tensor = torch.flip(bin_tensor, dims=(1,)) # Flip horizontally

    axial_start = exclude_slices[0]
    axial_end = axial_size - exclude_slices[1]
    bin_tensor = bin_tensor[axial_start:axial_end,:,:,:]

    ## Perform Optional Transformations ##
    if shift_x!=0:
        bin_tensor = torch.roll(bin_tensor, shift_x, dims=(3,)) # Cycle (or 'Roll') tensor
    if shift_y!=0:
        bin_tensor = torch.roll(bin_tensor, -shift_y, dims=(2,)) # Cycle (or 'Roll') tensor

    if crop_size!=-1: # Crop before resize
        bin_tensor = crop_image_tensor_by_size(bin_tensor, crop_size=crop_size)

    if resize_size!=-1:
        bin_tensor = transforms.Resize(size = (resize_size, resize_size), antialias=True)(bin_tensor) # Resize tensor

    if blur!=-1:
        bin_tensor = transforms.GaussianBlur(blur[0], sigma=(blur[1]))(bin_tensor)

    if normalize_scale!=-1:
        batch_size = len(bin_tensor)
        image_size_y = bin_tensor.shape[2]
        image_size_x = bin_tensor.shape[3]
        a = torch.reshape(bin_tensor,(batch_size, 1, image_size_x*image_size_y)) # Flattens each image
        a = nn.functional.normalize(a, p=1, dim = 2)
        a = torch.reshape(a,(batch_size, 1 , image_size_y, image_size_x)) # Reshapes images back into square matrices
        bin_tensor = normalize_scale*a

    if numpy==True:
        output = bin_tensor.numpy()
    else:
        output = bin_tensor.to(device)

    return header_info, output

## Compile Single Data Type

In [ ]:
def compile_single_data_type(load_dir_path, data_filename_stem, start_section_num, end_section_num, overlap=12, num_output_channels=1, shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,
                             normalize_scale=-1, visualize=False):
    '''
    This function loads a single file type for each section of the dataset. This type can be images (reconstructed activity, attenuation map, etc.) or sinograms.
    The function then contatenates each of the sections for this single file type to create a larger numpy array.

    load_dir_path:         Directory from which to load data
    data_filename_stem:    These will be filenames but without the numbers (indicating section numbers). It does include the extension.
                           For example, if rebin_sino_stem='rebinned_sino.hs', the section names will go from "rebinned_sino{start_section_num}.hs" to 'rebinned_sino{end_sections_num}.hs'
    start_section_num:     Start compilation with this section number.
    end_section_num:       End compilation with this section number.
    overlap:               The overlap between adjacent sections. This was chosen when creating the dataset. For our dataset, overlap=12.
    num_output_channels:          Number of channels. If >1, a sliding window is used to place adjacent in each example. For example, if num_output_channels=3, the Nth training example will have 3 channels.
                               The channel 0 will contain the N-1 slice, channel 1 the Nth slice, and channel 2 the N+1 slice. This is usually done for sinograms.
    shift_x:               Shift data left or right. Only used on images to align reconstructions with ground truths.
    shift_y:               Shift data up or down. Only used on images
    crop_size:             Crop data. Used on images to ensure ground truths & reconstructions are the same size. Set to -1 to not crop.
    resize_size:           Resize data. Used to resize reconstructions so that they are the same size as the ground truths maps.
    '''

    for section_num in range(start_section_num, end_section_num+1): # loop over the phantom sections. Remember, Python ends a for loop at one less than (end_section+1).

        # Construct header filename
        data_section_header_filename =os.path.splitext(data_filename_stem)[0] +str(section_num) +os.path.splitext(data_filename_stem)[1] # filename = stem + section_num + extension

        # Grab the data sections. Returned header dictionaries are discarded.
        h, section_arr = read_interfile_data(load_dir_path, data_section_header_filename, exclude_slices=(0,0), numpy=True, shift_x=shift_x, shift_y=shift_y,
                                             crop_size=crop_size, resize_size=resize_size)

        # If necessary, add channels by stacking slices (usually done for sinograms)
        start_idx = int(overlap/2)+1
        end_idx = section_arr.shape[0]-int(overlap/2)

        if num_output_channels==1:
            section_arr = section_arr[start_idx:end_idx+1,:]
        else:
            section_arr = make_slice_stack(section_arr, start_idx, end_idx, num_output_channels)

        if section_num==start_section_num:
            phantom_array = section_arr
        else:
            phantom_array=np.append(phantom_array, section_arr, axis=0)

        if visualize==True:
            section_tensor = torch.from_numpy(section_arr)
            print('data_filename_stem : ', data_filename_stem)
            print('section_num: ', section_num)
            print('section shape: ', section_tensor.shape)
            show_single_unmatched_tensor(section_tensor)

    return phantom_array

'''
## Changeable Variables ##
load_dir_path='./dataset-males/XCAT_male_pt77'  # end_section = 16
map_stem='act_map_toGATE_.h33'
start_section_num=0
end_section_num=16
overlap=12    # For our dataset, the overlap is 12
num_output_channels=1

#load_dir_path='./dataset-males/XCAT_male_pt80' # end_section=15
#load_dir_path='./dataset-males/XCAT_male_pt92' # end_section=15

map_array = compile_single_data_type(load_dir_path, map_stem, start_section_num, end_section_num, overlap=overlap, num_output_channels=num_output_channels, shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,
                             normalize_scale=-1, visualize=False)
vis_tensor=torch.from_numpy(map_array[0:8])
show_single_unmatched_tensor(vis_tensor, fig_size=15)



map_array = compile_single_data_type(load_dir_path, map_stem, start_section_num, end_section_num, overlap=overlap, num_output_channels=3, shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,
                             normalize_scale=-1, visualize=False)
vis_tensor=torch.from_numpy(map_array[0:3])
show_single_unmatched_tensor(vis_tensor, fig_size=15)

#show_single_commonmap_tensor(torch.from_numpy(map_array), nrow=35, figsize=(35,47), cmap='inferno')
'''

"\n## Changeable Variables ##\nload_dir_path='./dataset-males/XCAT_male_pt77'  # end_section = 16\nmap_stem='act_map_toGATE_.h33'\nstart_section_num=0\nend_section_num=16\noverlap=12    # For our dataset, the overlap is 12\nnum_output_channels=1\n\n#load_dir_path='./dataset-males/XCAT_male_pt80' # end_section=15\n#load_dir_path='./dataset-males/XCAT_male_pt92' # end_section=15\n\nmap_array = compile_single_data_type(load_dir_path, map_stem, start_section_num, end_section_num, overlap=overlap, num_output_channels=num_output_channels, shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,\n                             normalize_scale=-1, visualize=False)\nvis_tensor=torch.from_numpy(map_array[0:8])\nshow_single_unmatched_tensor(vis_tensor, fig_size=15)\n\n\n\nmap_array = compile_single_data_type(load_dir_path, map_stem, start_section_num, end_section_num, overlap=overlap, num_output_channels=3, shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,\n                             normalize_scale

## Make Single Phantom Files

In [ ]:
def make_single_phantom_files(load_dir_path, start_section_num, end_section_num, overlap, num_sino_channels=5, normalize_scale=-1, visualize=False,
            rebin_lowCountSino_stem='rebin_lowCountSino_.hv', rebin_medCountSino_stem='rebin_medCountSino_.hv', rebin_highCountSino_stem='rebin_highCountSino_.hv',
            rebin_lowCountImage_stem='rebin_lowCountImage_.hv', rebin_medCountImage_stem='rebin_medCountImage_.hv', rebin_highCountImage_stem='rebin_highCountImage_.hv',
            oblique_image_stem='oblique_image_.hv', anni_map_stem='anni_map_fromGATE_.h33', atten_map_stem='atten_map_fromGATE_.h33', act_map_stem='act_map_toGATE_.h33',
            save_types=[False,False,True,False,False,True,True,True,True,True], save_dir_path='./', phantom_label=''):
    '''
    This function loads and compiles each section of a phantom for each datatype (maps, sinograms, etc.).
    By altering the code below, one can alter the shift_x, shift_y, crop_size & resize_size for each data type. This allows one to make sure
    that the reconstructed images line up perfectly on top of the attenuation map, annihilation map, and activity maps. Values that we hardcoded
    in this function work for our dataset.

    load_dir_path:             path to the directory with the data files.
    start_section_num:         the lowest numbered section that you will be concatenating.
    end_section_num:           the highest numbered section that you will be concatenating.
    overlap:                   overlap between sections. Note: slices are left alone on the first and last section because there is no overlap at the ends of the phantom.
    num_sino_channels:         number of channels for the stacked sinogram arrays.
    normalize_scale:           normalize each slice to this number. Set to -1 to not normalize.
    visualize:                 Set to True to visualize the phantom sections as they compile.
    rebin_lowCountSino_stem:   filename stem for the low-count rebinned sinograms. The sinogram section names will
                               consist of this stem with a number added to the end of the filename (but before
                               the extension). For example, if rebin_sino_stem='rebinned_sino.hs', the
                               section names will go from "rebinned_sino{start_section_num}.hs" to 'rebinned_sino{end_sections_num}.hs'
    rebin_medCountSino_stem    filename stem for the medium-count rebinned sinograms.
    rebin_highCountSino_stem   filename stem for the high-count rebinned sinograms.
    rebin_lowCountImage_stem:  filename stem for PET image reconstructed from the low-count rebinned sinogram
    rebin_medCountImage_stem:  filename stem for PET image reconstructed from the medium-count rebinned sinogrammax_axial_index
    rebin_highCountImage_stem: filename stem for PET image reconstructed from the high-count rebinned sinogram
    oblique_image_stem:        filename stem for images reconstructed from the (4-D) oblique sinograms
    anni_map_stem:             filename stem for the annihilation maps
    atten_map_stem:            filename stem for the attenuation maps
    act_map_stem:              filename stem for the activity maps
    save_types:                a list of boolean values corresponding to which data types to save. Order starts with low count sinograms and ends with activity maps (as above).
    save_dir_path              path to the save directory
    phantom_label:             a prefix to add to the front of each compiled section. This will usually be something like "male_pt77" or "female_pt80", etc.
    '''

    lowCountImage =compile_single_data_type(load_dir_path, rebin_lowCountImage_stem, start_section_num, end_section_num, overlap, 1,
                                            shift_x=0, shift_y=0, crop_size=251, resize_size=180,
                                            normalize_scale=normalize_scale, visualize=visualize)
    medCountImage =compile_single_data_type(load_dir_path, rebin_medCountImage_stem, start_section_num, end_section_num, overlap, 1,
                                            shift_x=0, shift_y=0, crop_size=251, resize_size=180,
                                            normalize_scale=normalize_scale, visualize=visualize)
    highCountImage=compile_single_data_type(load_dir_path, rebin_highCountImage_stem,start_section_num, end_section_num, overlap, 1,
                                            shift_x=0, shift_y=0, crop_size=251, resize_size=180,
                                            normalize_scale=normalize_scale, visualize=visualize)
    obliqueImage  =compile_single_data_type(load_dir_path, oblique_image_stem,       start_section_num, end_section_num, overlap, 1,
                                            shift_x=0, shift_y=0, crop_size=251, resize_size=180,
                                            normalize_scale=normalize_scale, visualize=visualize)

    lowCountSino  =compile_single_data_type(load_dir_path, rebin_lowCountSino_stem,  start_section_num, end_section_num, overlap, num_sino_channels,
                                            shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,
                                            normalize_scale=normalize_scale, visualize=visualize)
    medCountSino  =compile_single_data_type(load_dir_path, rebin_medCountSino_stem,  start_section_num, end_section_num, overlap, num_sino_channels,
                                            shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,
                                            normalize_scale=normalize_scale, visualize=visualize)
    highCountSino =compile_single_data_type(load_dir_path, rebin_highCountSino_stem, start_section_num, end_section_num, overlap, num_sino_channels,
                                            shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,
                                            normalize_scale=normalize_scale, visualize=visualize)

    anniMap       =compile_single_data_type(load_dir_path, anni_map_stem,  start_section_num, end_section_num, overlap, 1,
                                            shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,
                                            normalize_scale=normalize_scale, visualize=visualize)
    attenMap      =compile_single_data_type(load_dir_path, atten_map_stem, start_section_num, end_section_num, overlap, 1,
                                            shift_x=0, shift_y=0, crop_size=-1, resize_size=-1,
                                            normalize_scale=normalize_scale,visualize=visualize)
    # The activity map must be aligned with atten_map and anni_map, because while the latter are output by GATE the former is output by STIR. The two do not exactly align.
    actMap        =compile_single_data_type(load_dir_path, act_map_stem,   start_section_num, end_section_num, overlap, 1,
                                            shift_x=-4, shift_y=5, crop_size=-1, resize_size=-1,
                                            normalize_scale=normalize_scale, visualize=visualize)

    # Determine first and last indexes which have nonzero ground truth activity
    start_idx, end_idx = first_last_nonzero(actMap)

    # Slice off ends of arrays with zero ground truth activity.
    print('Slicing off ends of with zero activity')
    lowCountSino = lowCountSino[start_idx:end_idx+1,:]
    medCountSino = medCountSino[start_idx:end_idx+1,:]
    highCountSino = highCountSino[start_idx:end_idx+1,:]

    lowCountImage = lowCountImage[start_idx:end_idx+1,:]
    medCountImage = medCountImage[start_idx:end_idx+1,:]
    highCountImage= highCountImage[start_idx:end_idx+1,:]
    obliqueImage  = obliqueImage[start_idx:end_idx+1,:]

    anniMap =       anniMap[start_idx:end_idx+1,:]
    attenMap =      attenMap[start_idx:end_idx+1,:]
    actMap =        actMap[start_idx:end_idx+1,:]

    print('Saving files')
    # Construct the beginning of the path, including the file prefix
    save_begin=save_dir_path+'/'+phantom_label+'-'

    if save_types[0]==True:
        np.save(save_begin+'lowCountSino', lowCountSino)
    if save_types[1]==True:
        np.save(save_begin+'medCountSino', medCountSino)
    if save_types[2]==True:
        np.save(save_begin+'highCountSino', highCountSino)
    if save_types[3]==True:
        np.save(save_begin+'lowCountImage', lowCountImage)
    if save_types[4]==True:
        np.save(save_begin+'medCountImage', medCountImage)
    if save_types[5]==True:
        np.save(save_begin+'highCountImage', highCountImage)
    if save_types[6]==True:
        np.save(save_begin+'obliqueImage', obliqueImage)
    if save_types[7]==True:
        np.save(save_begin+'anniMap', anniMap)
    if save_types[8]==True:
        np.save(save_begin+'attenMap', attenMap)
    if save_types[9]==True:
        np.save(save_begin+'actMap', actMap)

    return lowCountSino, medCountSino, highCountSino, lowCountImage, medCountImage, highCountImage, obliqueImage, anniMap, attenMap, actMap

'''
## Changeable Variables ##
load_dir_path='./dataset-males/XCAT_male_pt77'  # end_section = 16
#load_dir_path='./dataset-males/XCAT_male_pt80' # end_section=15
#load_dir_path='./dataset-males/XCAT_male_pt92' # end_section=15
start_section_num=0
end_section_num=16
overlap=12    # For our dataset, the overlap is 12
num_sino_channels=5
normalize_scale=-1
visualize=False
save_dir_path='./compiled'
save_prefix='male_pt77'

lowCountSino, medCountSino, highCountSino, lowCountImage, medCountImage, highCountImage, obliqueImage, anniMap, attenMap, actMap = make_single_phantom_files(
                                load_dir_path, start_section_num, end_section_num, overlap, num_sino_channels,
                                normalize_scale=normalize_scale, visualize=visualize, save_dir_path=save_dir_path, save_prefix=save_prefix)

print('low/med/high count sinograms')
print(lowCountSino.shape)
print(medCountSino.shape)
print(highCountSino.shape)
#show_single_commonmap_tensor(torch.from_numpy(highCountSino), nrow=47, figsize=(47,47), cmap='inferno')
#show_single_unmatched_tensor(torch.from_numpy(highCountSino[500:502]), fig_size=40)

print('low/med/high count images')
print(lowCountImage.shape)
print(medCountImage.shape)
print(highCountImage.shape)
#show_single_commonmap_tensor(torch.from_numpy(highCountImage), nrow=47, figsize=(47,47), cmap='inferno')
#show_single_unmatched_tensor(torch.from_numpy(highCountImage[200:205]), fig_size=3)

print('oblique image')
print(obliqueImage.shape)
#show_single_commonmap_tensor(torch.from_numpy(obliqueImage), nrow=47, figsize=(47,47), cmap='inferno')
#show_single_unmatched_tensor(torch.from_numpy(obliqueImage[200:205]), fig_size=3)

print('anni/atten/act maps')
print(anniMap.shape)
print(attenMap.shape)
print(actMap.shape)

#show_single_commonmap_tensor(torch.from_numpy(attenMap), nrow=40, figsize=(47,47), cmap='inferno')
#show_single_commonmap_tensor(torch.from_numpy(actMap), nrow=47, figsize=(35,47), cmap='inferno')
#show_single_unmatched_tensor(torch.from_numpy(actMap[200:205]), fig_size=3)
'''

"\n## Changeable Variables ##\nload_dir_path='./dataset-males/XCAT_male_pt77'  # end_section = 16\n#load_dir_path='./dataset-males/XCAT_male_pt80' # end_section=15\n#load_dir_path='./dataset-males/XCAT_male_pt92' # end_section=15\nstart_section_num=0\nend_section_num=16\noverlap=12    # For our dataset, the overlap is 12\nnum_sino_channels=5\nnormalize_scale=-1\nvisualize=False\nsave_dir_path='./compiled'\nsave_prefix='male_pt77'\n\nlowCountSino, medCountSino, highCountSino, lowCountImage, medCountImage, highCountImage, obliqueImage, anniMap, attenMap, actMap = make_single_phantom_files(\n                                load_dir_path, start_section_num, end_section_num, overlap, num_sino_channels,\n                                normalize_scale=normalize_scale, visualize=visualize, save_dir_path=save_dir_path, save_prefix=save_prefix)\n\nprint('low/med/high count sinograms')\nprint(lowCountSino.shape)\nprint(medCountSino.shape)\nprint(highCountSino.shape)\n#show_single_commonmap_tenso

## Make Multiple Phantom Files

In [ ]:
def make_multiple_phantom_files(phantoms_dir_path, phantom_list, phantom_dir_prefix='XCAT_',
                                          start_section_num=0, overlap=12, num_sino_channels=5, normalize_scale=-1,
                                          save_types=[False,False,True,False,False,True,True,True,True,True], save_dir_path='./'):
    '''
    Construct and save numpy files for multiple phantoms (training set, test set, etc.) Arguments are defined as before, with the following new ones:

    phantoms_dir_path:       Path to the directory containing the phantoms directories. This is different than the load_dir_path, which would be a path to the files
                             within an individual phantom (in other words, the path to a single phantom directory).
    phantom_list:            List of the names of the phantom directories (without prefix) for which to construct the files.
    phantom_dir_prefix:      A prefix to place before entries in phantom_list which gives the full name of the phantom directories. This will usually be 'XCAT_'
    '''

    for phantom_name in phantom_list:

        phantom_dir_name=phantom_dir_prefix+phantom_name
        load_dir_path=phantoms_dir_path+'/'+phantom_dir_name
        start_section_num=start_section_num

        ## Determine start_section_num and end_section_num for a single phantom
        end_section_num=max_trailing_integer(load_dir_path)

        print('Phantom: ', phantom_dir_name)
        print('end_section_num: ', end_section_num)

        # Return numpy data files. We don't save these here because they are saved in construct_single_phantom_files()

        lowCountSino, medCountSino, highCountSino, lowCountImage, medCountImage, highCountImage, obliqueImage, anniMap, attenMap, actMap = make_single_phantom_files(
                                load_dir_path, start_section_num, end_section_num, overlap, num_sino_channels, normalize_scale=normalize_scale, visualize=False,
                                save_types=save_types, save_dir_path=save_dir_path, phantom_label=phantom_name)

## Changeable Variables ##
'''
train_codes=['male_pt96', 'male_pt108', 'male_pt118', 'male_pt139', 'male_pt141', 'male_pt144', 'male_pt145', 'male_pt155', 'male_pt157', 'male_pt163', 'male_pt164', 'male_pt167', 'male_pt169', 'male_pt171', 'male_pt178', 'male_pt180', 'male_pt184', 'male_pt196', 'male_pt200', 'male_pt168', 'female_pt71', 'female_pt76', 'female_pt86', 'female_pt89', 'female_pt98', 'female_pt99', 'female_pt117', 'female_pt152', 'female_pt153', 'female_pt166', 'female_pt175', 'female_pt176', 'female_pt182', 'female_pt170']
validation_codes=['male_pt77', 'male_pt80', 'male_pt92', 'male_pt150', 'male_pt154', 'male_pt173', 'female_pt140', 'female_pt142', 'female_pt143', 'female_pt151', 'female_pt162']
test_codes=['male_pt93', 'male_pt128', 'male_pt159', 'male_pt201', 'female_pt106', 'female_pt147', 'female_pt149']

phantoms_dir_path='./dataset-XCAT/'
phantom_dir_prefix='XCAT_'
save_dir_path='./compiled-test'
normalize_scale=1
num_sino_channels=3
save_types=[False,False,True,False,False,True,True,True,True,True]


phantom_list=test_codes
make_multiple_phantom_files(phantoms_dir_path, phantom_list, phantom_dir_prefix=phantom_dir_prefix,
                                      start_section_num=0, overlap=12, num_sino_channels=num_sino_channels, normalize_scale=-1,
                                      save_types=save_types, save_dir_path=save_dir_path)
'''

"\ntrain_codes=['male_pt96', 'male_pt108', 'male_pt118', 'male_pt139', 'male_pt141', 'male_pt144', 'male_pt145', 'male_pt155', 'male_pt157', 'male_pt163', 'male_pt164', 'male_pt167', 'male_pt169', 'male_pt171', 'male_pt178', 'male_pt180', 'male_pt184', 'male_pt196', 'male_pt200', 'male_pt168', 'female_pt71', 'female_pt76', 'female_pt86', 'female_pt89', 'female_pt98', 'female_pt99', 'female_pt117', 'female_pt152', 'female_pt153', 'female_pt166', 'female_pt175', 'female_pt176', 'female_pt182', 'female_pt170']\nvalidation_codes=['male_pt77', 'male_pt80', 'male_pt92', 'male_pt150', 'male_pt154', 'male_pt173', 'female_pt140', 'female_pt142', 'female_pt143', 'female_pt151', 'female_pt162']\ntest_codes=['male_pt93', 'male_pt128', 'male_pt159', 'male_pt201', 'female_pt106', 'female_pt147', 'female_pt149']\n\nphantoms_dir_path='./dataset-XCAT/'\nphantom_dir_prefix='XCAT_'\nsave_dir_path='./compiled-test'\nnormalize_scale=1\nnum_sino_channels=3\nsave_types=[False,False,True,False,False,True,True

## Make Datasets from Multiple Phantoms

In [ ]:
from pathlib import Path
import numpy as np
from tqdm import tqdm
import os
from google.colab import drive
import time

def compile_phantoms_chunked(save_dir_path, save_name, load_dir_path, phantom_list, chunk_size=None, use_colab=True):
    """
    Stitch multiple phantom .npy files into single .npy files on Google Drive (chunked, low RAM).
    Does everything on google drive so may be a bit slower.

    Args:
        save_dir_path: path to folder to save compiled arrays
        save_name: base name for output files
        load_dir_path: path to folder containing phantom .npy files
        phantom_list: list of phantom prefixes (without file type or .npy)
        chunk_size: optional number of examples to copy at a time (reduces RAM)
        use_colab: if True, mount Google Drive
    """
    load_dir_path = Path(load_dir_path)
    save_dir_path = Path(save_dir_path)
    save_dir_path.mkdir(parents=True, exist_ok=True)

    if use_colab:
        drive.mount('/content/drive', force_remount=True)

    # Detect file types from the first phantom
    first_phantom = phantom_list[0]
    detected_types = []
    for fpath in load_dir_path.glob(f"{first_phantom}-*.npy"):
        fname = fpath.name
        type_str = fname[len(first_phantom)+1:-4]
        detected_types.append(type_str)
    file_types = sorted(detected_types)
    print(f"Detected file types: {file_types}")

    for ftype in file_types:
        final_file_path = save_dir_path / f"{save_name}-{ftype}.npy"

        if final_file_path.exists():
            print(f"[SKIPPED] '{final_file_path}' already exists")
            continue

        # Step 1: compute final shape and dtype
        total_examples = 0
        base_shape = None
        dtype = None
        files = sorted(load_dir_path.glob(f"*-{ftype}.npy"))
        for f in files:
            arr = np.load(f, mmap_mode="r")
            total_examples += arr.shape[0]
            if base_shape is None:
                base_shape = arr.shape[1:]
                dtype = arr.dtype
            else:
                if arr.shape[1:] != base_shape:
                    raise ValueError(f"All arrays for '{ftype}' must match shape {base_shape} along axes 1+")
                if arr.dtype != dtype:
                    raise ValueError(f"All arrays for '{ftype}' must have same dtype {dtype}")
            del arr

        final_shape = (total_examples,) + base_shape

        # Step 2: create memmap directly on Google Drive
        output = np.lib.format.open_memmap(str(final_file_path), mode='w+', dtype=dtype, shape=final_shape)

        # Step 3: copy each file in chunks
        offset = 0
        for f in tqdm(files, desc=f"Stitching {ftype}"):
            arr = np.load(f, mmap_mode="r")
            n_examples = arr.shape[0]

            # Determine chunk size
            if chunk_size is None:
                # Copy the whole file at once
                output[offset:offset+n_examples] = arr
            else:
                for start in range(0, n_examples, chunk_size):
                    end = min(start + chunk_size, n_examples)
                    output[offset+start:offset+end] = arr[start:end]

            offset += n_examples
            del arr

        output.flush()
        del output
        print(f"[DONE] '{final_file_path}' saved with shape {final_shape}")



def compile_phantoms_memmap(save_dir_path, save_name, load_dir_path, phantom_list, use_colab=True, max_retries=3):
    """
    Stitch multiple phantom .npy files into memmaps by file type, memory-efficiently,
    skipping already existing compiled files in save_dir_path, with robust Google Drive copy,
    and consistent progress display for skipped and processed file types.

    Uses the VM to speed things along. Will be limited by drive space in the VM.
    The next function does everything on Google Drive (may be slower)
    """
    load_dir_path = Path(load_dir_path)
    save_dir_path = Path(save_dir_path)
    save_dir_path.mkdir(parents=True, exist_ok=True)

    # Mount Google Drive if using Colab
    if use_colab:
        drive.mount('/content/drive', force_remount=True)
        work_output_dir = Path("/content/tmp_compiled")
        work_output_dir.mkdir(parents=True, exist_ok=True)
    else:
        work_output_dir = save_dir_path
        work_output_dir.mkdir(parents=True, exist_ok=True)

    # Detect file types from the first phantom
    first_phantom = phantom_list[0]
    detected_types = []
    for fpath in load_dir_path.glob(f"{first_phantom}-*.npy"):
        fname = fpath.name
        type_str = fname[len(first_phantom)+1:-4]  # strip "phantom-" prefix and ".npy"
        detected_types.append(type_str)
    file_types = sorted(detected_types)
    print(f"Detected file types: {file_types}")

    # Helper function to copy with retry
    def copy_with_retry(src_path, dst_path, retries=max_retries):
        for attempt in range(retries):
            try:
                buffer_size = 1024 * 1024
                total_size = os.path.getsize(src_path)
                with open(src_path, "rb") as src, open(dst_path, "wb") as dst:
                    with tqdm(total=total_size, unit='B', unit_scale=True, desc=f"Copying {dst_path.name}") as pbar:
                        while True:
                            buf = src.read(buffer_size)
                            if not buf:
                                break
                            dst.write(buf)
                            pbar.update(len(buf))
                return
            except OSError as e:
                print(f"Copy attempt {attempt+1} failed: {e}")
                if attempt < retries - 1:
                    print("Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    raise e

    # Process each file type
    for ftype in file_types:
        final_file_path = save_dir_path / f"{save_name}-{ftype}.npy"

        if final_file_path.exists():
            with tqdm(total=1, desc=f"Skipping {ftype}", leave=False) as pbar:
                pbar.update(1)
            print(f"[SKIPPED] File '{final_file_path}' already exists")
            continue

        # Gather all files of this type
        files = sorted(load_dir_path.glob(f"*-{ftype}.npy"))

        # Step 1: determine total size and shape
        total_examples = 0
        base_shape = None
        dtype = None
        for f in files:
            arr = np.load(f, mmap_mode="r")
            total_examples += arr.shape[0]
            if base_shape is None:
                base_shape = arr.shape[1:]
                dtype = arr.dtype
            else:
                if arr.shape[1:] != base_shape:
                    raise ValueError(f"All arrays for type '{ftype}' must match shape {base_shape} along axes 1+")
                if arr.dtype != dtype:
                    raise ValueError(f"All arrays for type '{ftype}' must have the same dtype {dtype}")
            del arr

        # Step 2: create memmap in VM
        final_shape = (total_examples,) + base_shape
        memmap_path = work_output_dir / f"{save_name}-{ftype}.npy"
        output = np.lib.format.open_memmap(str(memmap_path), mode="w+", dtype=dtype, shape=final_shape)

        # Step 3: fill memmap one file at a time
        offset = 0
        for f in tqdm(files, desc=f"Stitching {ftype}"):
            arr = np.load(f, mmap_mode="r")
            n = arr.shape[0]
            output[offset:offset+n] = arr
            offset += n
            del arr

        # Step 4: flush and close memmap
        output.flush()
        del output
        print(f"[DONE] Completed '{ftype}' with shape {final_shape}")

        # Step 5: copy memmap to Drive if in Colab
        if use_colab:
            copy_with_retry(memmap_path, final_file_path)

            # Step 6: remove local VM file to free space
            os.remove(memmap_path)
            print(f"[CLEANUP] Deleted local memmap '{memmap_path}' to free VM space")
        else:
            # On local system, just rename/move memmap to final path
            memmap_path.rename(final_file_path)



train_codes=['male_pt96', 'male_pt108', 'male_pt118', 'male_pt139', 'male_pt141', 'male_pt144', 'male_pt145', 'male_pt155', 'male_pt157', 'male_pt163', 'male_pt164', 'male_pt167', 'male_pt169', 'male_pt171', 'male_pt178', 'male_pt180', 'male_pt184', 'male_pt196', 'male_pt200', 'male_pt168', 'female_pt71', 'female_pt76', 'female_pt86', 'female_pt89', 'female_pt98', 'female_pt99', 'female_pt117', 'female_pt152', 'female_pt153', 'female_pt166', 'female_pt175', 'female_pt176', 'female_pt182', 'female_pt170']
test_codes=['male_pt77', 'male_pt80', 'male_pt92', 'male_pt150', 'male_pt154', 'male_pt173', 'female_pt140', 'female_pt142', 'female_pt143', 'female_pt151', 'female_pt162']
val_codes=['male_pt93', 'male_pt128', 'male_pt159', 'male_pt201', 'female_pt106', 'female_pt147', 'female_pt149']
short_codes =['male_pt96', 'male_pt108']

save_dir_path = '/content/drive/MyDrive/Colab/Working/dataset/sets/'


load_dir_path = '/content/drive/MyDrive/Colab/Working/dataset/phantoms-test'
save_name = "test"
phantom_list = test_codes

compile_phantoms_memmap(save_dir_path, save_name, load_dir_path, phantom_list, use_colab=True)

Mounted at /content/drive
Detected file types: ['actMap', 'anniMap', 'attenMap', 'highCountImage', 'highCountSino', 'obliqueImage']


Stitching actMap: 100%|██████████| 11/11 [00:13<00:00,  1.26s/it]


[DONE] Completed 'actMap' with shape (5729, 1, 180, 180)


Copying test-actMap.npy: 100%|██████████| 742M/742M [00:01<00:00, 444MB/s]


[CLEANUP] Deleted local memmap '/content/tmp_compiled/test-actMap.npy' to free VM space


...

[CLEANUP] Deleted local memmap '/content/tmp_compiled/test-highCountSino.npy' to free VM space


Stitching obliqueImage: 100%|██████████| 11/11 [00:35<00:00,  3.22s/it]


[DONE] Completed 'obliqueImage' with shape (5729, 1, 180, 180)


Copying test-obliqueImage.npy: 100%|██████████| 742M/742M [00:02<00:00, 365MB/s]


[CLEANUP] Deleted local memmap '/content/tmp_compiled/test-obliqueImage.npy' to free VM space


In [ ]:
f = Path("/content/drive/MyDrive/Colab/Working/dataset/sets/train-highCountSino.npy")
print(f.exists(), f.stat().st_size / 1e9, "GB")

True 41.928885488 GB


# Sample, Trim, Resize Dataset

In [27]:
import numpy as np
import torch
import torchvision.transforms as T
from tqdm import tqdm
import os
import json
import shutil

import numpy as np
import torch
import torchvision.transforms as T
from tqdm import tqdm
import os
import json
import shutil

def sample_trim_resize(
    input_dir,
    input_file,
    output_dir,
    output_file,
    sample_division=1,
    remove_n=0,
    new_height=None,
    new_width=None,
    seed=42,
    log_file='log-SampleTrimResize.txt',
    skip_if_exists_vm=False
):
    """
    Sample, optionally remove slices, resize, and copy to Google Drive with monitoring.
    Automatically verifies that the copied file matches the VM file.

    Args:
        input_dir: Path to input directory containing numpy file
        input_file: Name of input numpy file
        output_dir: Path to directory to save output (Google Drive)
        output_file: Name of output file
        sample_division: Keep every N-th slice
        remove_n: Number of slices to randomly remove before sampling
        new_height, new_width: Resize dimensions; if None, original used
        seed: Random seed for reproducibility
        log_file: Name of log file in current working directory
        skip_if_exists_vm: If True and the VM file exists, skip processing slices
    """

    # --- Clean filenames and paths ---
    input_file = input_file.lstrip('/')
    output_file = output_file.lstrip('/')
    input_path = os.path.join(input_dir, input_file)
    local_output_path = os.path.join("/content", output_file)
    drive_output_path = os.path.join(output_dir, output_file)
    os.makedirs(output_dir, exist_ok=True)

    # --- Load input as memory-mapped array ---
    arr = np.load(input_path, mmap_mode='r')
    n_samples, channels, height, width = arr.shape

    # --- RNG setup ---
    rng = np.random.default_rng(seed)

    # --- Random slice removal ---
    all_indices = np.arange(n_samples)
    keep_mask = np.ones(n_samples, dtype=bool)
    if remove_n > 0:
        remove_idx = rng.choice(n_samples, size=remove_n, replace=False)
        keep_mask[remove_idx] = False
    remaining_idx = all_indices[keep_mask]

    # --- Even sampling ---
    sampled_idx = remaining_idx[::sample_division]
    n_final = len(sampled_idx)

    out_h = new_height if new_height is not None else height
    out_w = new_width if new_width is not None else width

    # --- GPU device ---
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # --- Prepare transform if resizing ---
    resize_transform = None
    if new_height is not None or new_width is not None:
        resize_transform = T.Resize((out_h, out_w), antialias=True)

    # --- Process slices if VM file doesn't exist or skipping disabled ---
    if skip_if_exists_vm and os.path.exists(local_output_path):
        print(f"VM file already exists: {local_output_path}. Skipping processing slices...")
    else:
        result = np.lib.format.open_memmap(local_output_path, mode='w+', dtype=np.float32,
                                          shape=(n_final, channels, out_h, out_w))
        tqdm_bar = tqdm(sampled_idx, desc="Processing slices")
        for i, idx in enumerate(tqdm_bar):
            slice_i = arr[idx]  # [C,H,W]
            if resize_transform:
                slice_tensor = torch.from_numpy(slice_i).to(device)
                slice_tensor = resize_transform(slice_tensor)
                slice_i = slice_tensor.cpu().numpy().astype(np.float32)
            result[i] = slice_i

        # Flush memory-mapped array
        print("Flushing memory-mapped array to disk...")
        result.flush()

    # --- Copy to Drive if paths differ ---
    if os.path.abspath(local_output_path) != os.path.abspath(drive_output_path):
        print(f"Copying output to Drive: {drive_output_path}")
        shutil.copy(local_output_path, drive_output_path)
    else:
        print("Source and destination are the same — skipping copy.")

    # --- Verify copied file ---
    vm_size = os.path.getsize(local_output_path)
    drive_size = os.path.getsize(drive_output_path)

    vm_arr = np.load(local_output_path, mmap_mode='r')
    drive_arr = np.load(drive_output_path, mmap_mode='r')

    if (vm_size == drive_size and
        vm_arr.shape == drive_arr.shape and
        vm_arr.dtype == drive_arr.dtype):
        print(f"\n✅ File verification successful! VM and Drive files match.")
        print(f"   Shape: {vm_arr.shape}, dtype: {vm_arr.dtype}, size: {vm_size/1e6:.2f} MB")
    else:
        print("\n⚠️ File verification failed! VM and Drive files differ.")
        print(f"   VM -> shape: {vm_arr.shape}, dtype: {vm_arr.dtype}, size: {vm_size/1e6:.2f} MB")
        print(f"   Drive -> shape: {drive_arr.shape}, dtype: {drive_arr.dtype}, size: {drive_size/1e6:.2f} MB")

    # --- Log everything ---
    run_entry = {
        "input_dir": input_dir,
        "input_file": input_file,
        "output_dir": output_dir,
        "output_file": output_file,
        "sample_division": sample_division,
        "remove_n": remove_n,
        "seed": seed,
        "new_height": new_height,
        "new_width": new_width,
        "output_shape": vm_arr.shape
    }
    log_path = os.path.join(os.getcwd(), log_file)
    with open(log_path, 'a') as f:
        f.write(json.dumps(run_entry) + "\n")

    print(f"   Log updated: {os.path.abspath(log_file)}")



sample_trim_resize(
    input_dir='/content/drive/MyDrive/Colab/Working/dataset/sets',
    input_file='train-highCountSino-382x513.npy',
    output_dir='/content/drive/MyDrive/Colab/Working/dataset/sets',
    output_file='train-highCountSino-180x180.npy',
    sample_division=1,
    remove_n=0,
    new_height=180,
    new_width=180,
    seed=0,                # default seed for reproducibility
    log_file="log-SampleTrimResize.txt"
    skip_if_exists_vm=False
)


Using device: cpu


Processing slices: 100%|██████████| 17830/17830 [04:52<00:00, 61.05it/s]


Flushing memory-mapped array to disk...
Copying output to Drive: /content/drive/MyDrive/Colab/Working/dataset/sets/train-highCountSino-180x180.npy

✅ File verification successful! VM and Drive files match.
   Shape: (17830, 3, 180, 180), dtype: float32, size: 6932.30 MB
   Log updated: /content/log-SampleTrimResize.txt


# New Section

# Investigate

## Look at Dataset Shapes

In [13]:
## SET VARIABLES ##

config={'SI_normalize':False, 'SI_scale':1}
exclude_slices=(5,5)
fig_size=4
num_cols=3
crop_size=260
resize_size=180
normalize_scale=1


load_dir_path = '/content/drive/MyDrive/Colab/Working/dataset/sets/'
set_name = 'test-highCountSino.npy'
load_dir = f"{load_dir_path}{set_name}"
large_array=np.load(load_dir, mmap_mode='r')

print(large_array.shape)


(5729, 3, 382, 513)


## Look at ROIs

In [ ]:
'''
header_name_act2='PhantomAct2_act.h33'
header_name_act4='PhantomAct4_act.h33'
header_name_mask_background='PhantomMaskBackground17_act.h33'
header_name_mask_hot='PhantomMaskHot17_act.h33'

exclude_slices=(0,0)
mask_dir_path='../STIR-GATE-Connection-simplified/QAPhantoms/Masks/'
load_dir_path='../STIR-GATE-Connection-simplified/QAPhantoms/FinishedPhantoms/'


header_act2, tensor_act2                      = read_interfile_data(load_dir_path, header_name_act2,            exclude_slices=exclude_slices)
header_act4, tensor_act4                      = read_interfile_data(load_dir_path, header_name_act4,            exclude_slices=exclude_slices)
header_mask_background, tensor_mask_background= read_interfile_data(mask_dir_path, header_name_mask_background, exclude_slices=exclude_slices)
header_mask_hot, tensor_mask_hot              = read_interfile_data(mask_dir_path, header_name_mask_hot,        exclude_slices=exclude_slices)

x=ROI_NEMA_hot(tensor_act2, tensor_act4, tensor_mask_background, tensor_mask_hot)

print(x)

#show_single_unmatched_tensor(tensor_act2)
#show_single_commonmap_tensor(tensor_act2)
#show_single_unmatched_tensor(tensor_act4)
#show_single_commonmap_tensor(tensor_act4)
'''

"\nheader_name_act2='PhantomAct2_act.h33'\nheader_name_act4='PhantomAct4_act.h33'\nheader_name_mask_background='PhantomMaskBackground17_act.h33'\nheader_name_mask_hot='PhantomMaskHot17_act.h33'\n\nexclude_slices=(0,0)\nmask_dir_path='../STIR-GATE-Connection-simplified/QAPhantoms/Masks/'\nload_dir_path='../STIR-GATE-Connection-simplified/QAPhantoms/FinishedPhantoms/'\n\n\nheader_act2, tensor_act2                      = read_interfile_data(load_dir_path, header_name_act2,            exclude_slices=exclude_slices)\nheader_act4, tensor_act4                      = read_interfile_data(load_dir_path, header_name_act4,            exclude_slices=exclude_slices)\nheader_mask_background, tensor_mask_background= read_interfile_data(mask_dir_path, header_name_mask_background, exclude_slices=exclude_slices)\nheader_mask_hot, tensor_mask_hot              = read_interfile_data(mask_dir_path, header_name_mask_hot,        exclude_slices=exclude_slices)\n\nx=ROI_NEMA_hot(tensor_act2, tensor_act4, tensor_

## Inspect Alignment

In [ ]:
## CHECK ALIGNMENT ##
'''
fig_size=3
start_index=400
end_index=403
mult_factor=100000

anniMap=torch.from_numpy(np.load('./compiled/male_pt96-anniMap.npy', mmap_mode='r'))
actMap=torch.from_numpy(np.load('./compiled/male_pt96-actMap.npy', mmap_mode='r'))
attenMap=torch.from_numpy(np.load('./compiled/male_pt96-attenMap.npy', mmap_mode='r'))

obliqueImage=torch.from_numpy(np.load('./compiled/male_pt96-obliqueImage.npy', mmap_mode='r'))
highCountSino=torch.from_numpy(np.load('./compiled/male_pt96-highCountSino.npy', mmap_mode='r'))
highCountImage=torch.from_numpy(np.load('./compiled/male_pt96-highCountImage.npy', mmap_mode='r'))

print('####highCountSino')
print(highCountSino.size())
print(highCountSino.sum())
print('####highCountImage')
print(highCountImage.size())
print(highCountImage.sum()/180**2)
print('####obliqueImage')
print(obliqueImage.size())
print(obliqueImage.sum()/180**2)
print('####actMap')
print(actMap.size())
print(actMap.sum()/180**2)
print('####anniMap')
print(anniMap.size())
print(anniMap.sum()/180**2)
print('####attenMap')
print(attenMap.size())
print(attenMap.sum()/180**2)



show_single_unmatched_tensor(highCountSino[1:4], fig_size=12)

tensor_sum0=torch.add(actMap[start_index:end_index], mult_factor*highCountImage[start_index:end_index])
tensor_sum1=torch.add(actMap[start_index:end_index], mult_factor*obliqueImage[start_index:end_index])
tensor_sum2=torch.add(actMap[start_index:end_index], mult_factor*attenMap[start_index:end_index])
tensor_sum3=torch.add(actMap[start_index:end_index], mult_factor*anniMap[start_index:end_index])
tensor_sum4=torch.add(actMap[start_index:end_index], mult_factor*actMap[start_index:end_index])

print('Lining Images Up on Top of Each Other')
print('=====================================')
show_single_unmatched_tensor(tensor_sum0, fig_size=fig_size)
show_single_unmatched_tensor(tensor_sum1, fig_size=fig_size)
show_single_unmatched_tensor(tensor_sum2, fig_size=fig_size)
show_single_unmatched_tensor(tensor_sum3, fig_size=fig_size)
show_single_unmatched_tensor(tensor_sum4, fig_size=fig_size)
'''

SyntaxError: incomplete input (ipython-input-4250258164.py, line 52)

In [ ]:
## CHECK ALIGNMENT ##
fig_size=3
start_index=400
end_index=403
mult_factor=500

actMapMM =np.load('/content/drive/MyDrive/Colab/Working/dataset/sets/train-actMap.npy', mmap_mode='r')
attenMapMM=np.load('/content/drive/MyDrive/Colab/Working/dataset/sets/train-attenMap.npy', mmap_mode='r')

actMap=torch.from_numpy(actMapMM[start_index:end_index,:])
attenMap=torch.from_numpy(attenMapMM[start_index:end_index,:])

print('####actMap')
print(actMap.size())
print(actMap.sum()/180**2)
print('####attenMap')
print(attenMap.size())
print(attenMap.sum()/180**2)

tensor_sum0=torch.add(actMap, mult_factor*attenMap)

print('Lining Images Up on Top of Each Other')
print('=====================================')
show_single_unmatched_tensor(tensor_sum0, fig_size=fig_size)

In [ ]:
from pathlib import Path
import numpy as np

# Path to the folder containing your compiled files
compiled_dir = Path("/content/drive/MyDrive/Colab/Working/dataset/sets")

# List all .npy compiled files
compiled_files = sorted(compiled_dir.glob("*.npy"))

# Open each file as a memmap and print its shape
for fpath in compiled_files:
    print(f"File: {fpath.name}")
    # open as memmap (read-only, minimal RAM usage)
    mm = np.load(fpath, mmap_mode='r')
    print(f"  Shape: {mm.shape}, dtype: {mm.dtype}")

    show_single_unmatched_tensor(torch.from_numpy(mm[300:305]), fig_size=5)
    del mm  # release memmap

## Visualize Interfiles

In [ ]:
## NOTES ##
# reconstruction shapes = 337x337
# sinogram shapes = 382x513

## SET VARIABLES ##

'''
config={'SI_normalize':False, 'SI_scale':1}
exclude_slices=5
fig_size=3
num_cols=3
normalize_scale=1

dataset_path='./dataset-males/'
phantom_dir='XCAT_male_pt77' # 71 76 86 89 98 99 106 117 140 142 143 147
section_num='0'


## LOAD INTERFILES
phantom_path=os.path.join(dataset_path, phantom_dir)

# The ground truth activity is input to GATE.
header_info, activity = read_interfile_data(phantom_path, 'act_map_toGATE_'+section_num+'.h33', exclude_slices=exclude_slices,
                                            shift_x=-4, shift_y=5, crop_size=-1, resize_size=-1, normalize_scale=normalize_scale)
# Attenuation is exported by GATE and input to STIR. I take this as the ground truth geometry.
header_info, atten =   read_interfile_data(phantom_path, 'atten_map_fromGATE_'+section_num+'.h33', exclude_slices=exclude_slices,
                                           shift_x=0, shift_y=0, crop_size=-1, resize_size=-1, normalize_scale=normalize_scale)
header_info, anni =   read_interfile_data(phantom_path, 'anni_map_fromGATE_'+section_num+'.h33', exclude_slices=exclude_slices,
                                           shift_x=0, shift_y=0, crop_size=-1, resize_size=-1, normalize_scale=normalize_scale)

# Reconstructions are both cropped and resized but not shifted (since it is exported by STIR, it lines up with the attenuation map)
header_info, highCountImage = read_interfile_data(phantom_path, 'rebin_highCountImage_'+section_num+'.hv', exclude_slices=exclude_slices,
                                                  shift_x=0, shift_y=0, crop_size=251, resize_size=180, normalize_scale=normalize_scale)
header_info, oblique = read_interfile_data(phantom_path, 'oblique_image_'+section_num+'.hv', exclude_slices=exclude_slices,
                                           shift_x=0, shift_y=0, crop_size=251, resize_size=180, normalize_scale=normalize_scale)

header_info, sino = read_interfile_data(phantom_path, 'rebin_highCountSino_'+section_num+'.hv', exclude_slices=exclude_slices,
                                        shift_x=0, shift_y=0, crop_size=251, resize_size=180, normalize_scale=-1)

#reconstructed = reconstruct(sino[0:num_cols], config, image_size=resize_size, recon_type='MLEM', circle=False)
#reconstructed = reconstruct(sino[0:num_cols], config, image_size=resize_size, recon_type='FBP', circle=False)

## CALCULATE METRICS ##

MSE_highCountImage=calculate_metric(activity, highCountImage, MSE, return_dataframe=False, label='default', crop_factor=1)
MSE_oblique=calculate_metric(activity, oblique, MSE, return_dataframe=False, label='default', crop_factor=1)

SSIM_rebin=calculate_metric(activity, highCountImage, SSIM, return_dataframe=False, label='default', crop_factor=1)
SSIM_oblique=calculate_metric(activity, oblique, SSIM, return_dataframe=False, label='default', crop_factor=1)

print('MSE (rebin): ', MSE_highCountImage)
print('MSE (oblique): ', MSE_oblique)
print('SSIM (rebin): ', SSIM_rebin)
print('SSIM (oblique): ', SSIM_oblique)

print('ATTENUATION')
show_single_unmatched_tensor(atten[0:num_cols], fig_size=fig_size)
print('GROUND TRUTH')
show_single_unmatched_tensor(activity[0:num_cols], fig_size=fig_size)
print('FORE RECON')
show_single_unmatched_tensor(highCountImage[0:num_cols], fig_size=fig_size)
print('OBLIQUE RECON')
show_single_unmatched_tensor(oblique[0:num_cols], fig_size=fig_size)
'''
'''
## CHECK ALIGNMENT ##
tensor_sum0=torch.add(anni[0:num_cols], atten[0:num_cols])
tensor_sum1=torch.add(activity[0:num_cols],atten[0:num_cols])
tensor_sum2=torch.add(activity[0:num_cols],2*highCountImage[0:num_cols])
tensor_sum3=torch.add(activity[0:num_cols],2*oblique[0:num_cols])
tensor_sum4=torch.add(oblique[0:num_cols],atten[0:num_cols])
#tensor_sum3=torch.add(activity[0:num_cols],reconstructed[0:num_cols])

print('Lining Images Up on Top of Each Other')
print('=====================================')
show_single_unmatched_tensor(tensor_sum0, fig_size=fig_size)
show_single_unmatched_tensor(tensor_sum1, fig_size=fig_size)
show_single_unmatched_tensor(tensor_sum2, fig_size=fig_size)
show_single_unmatched_tensor(tensor_sum3, fig_size=fig_size)
show_single_unmatched_tensor(tensor_sum4, fig_size=fig_size)
'''

# Classes

## Dataset

In [ ]:
def NpArrayDataLoader(image_array, sino_array, config, image_size = 90, sino_size=90, image_channels=1, sino_channels=1, augment=False, index=0):
    '''
    Function to load an image and a sinogram. Returns 4 pytorch tensors: the original dataset sinogram and image,
    and scaled and normalized sinograms and images.

    image_array:    image numpy array
    sino_array:     sinogram numpy array
    config:         configuration dictionary with hyperparameters
    image_size:     shape to resize image to (for output)
    image_channels: number of channels for output images
    sino_size:      shape to resize sinograms to (for output)
    sino_channels:  number of channels in output sinograms (for photopeak sinograms, this is 1)
    augment:        perform data augmentation?
    index:          index of the image/sinogram to grab
    '''
    ## Set Normalization Variables ##
    if (train_type=='GAN') or (train_type=='SUP'):
        if train_SI==True:
            SI_normalize=config['SI_normalize']
            SI_scale=config['SI_scale']
            IS_normalize=False     # If the Sinogram-->Image network (SI) is being trained, don't waste time normalizing sinograms
            IS_scale=1             # If the Sinogram-->Image network (SI) is being trained, don't waste time scaling sinograms
        else:
            IS_normalize=config['IS_normalize']
            IS_scale=config['IS_scale']
            SI_normalize=False
            SI_scale=1
    else:
        IS_normalize=config['IS_normalize']
        SI_normalize=config['SI_normalize']
        IS_scale=config['IS_scale']
        SI_scale=config['SI_scale']

    ## Data Augmentation Functions ##
    def RandRotate(image_2D, sinogram_3D):

        def IntersectCircularBorder(image):
            '''
            Function for determining whether an image itersects a circular boundary inscribed within the square FOV.
            This function is not currently used.
            '''
            y_max = image.shape[1]
            x_max = image.shape[2]

            r_max = y_max/2.0
            x_center = (x_max-1)/2.0 # the -1 comes from the fact that the coordinates of a pixel start at 0, not 1
            y_center = (y_max-1)/2.0

            margin_sum = 0
            for y in range(0, y_max):
                for x in range(0, x_max):
                    if r_max < ((x-x_center)**2 + (y-y_center)**2)**0.5 :
                        margin_sum += torch.sum(image[:,y,x]).item()

            return_value = True if margin_sum == 0 else False
            return return_value

        def IntersectSquareBorder(image):
            '''
            Function for determining whether the image intersects the edge of the square FOV. If it does not, then the image
            is fully specified by the sinogram, and data augmentation can be performed. If the image does
            intersect the edge of the image, then some of it may be cropped outside the FOV. In this case,
            augmentation via rotation should not be performed.
            '''
            max_idx = image.shape[1]-1
            margin_sum = torch.sum(image[:,0,:]).item() + torch.sum(image[:,max_idx,:]).item() \
                        +torch.sum(image[:,:,0]).item() + torch.sum(image[:,:,max_idx]).item()
            return_value = False if margin_sum == 0 else True
            return return_value

        if IntersectSquareBorder(image_2D) == False:
            bins = sinogram_3D.shape[1]                 # Sinogram is square, so sinogram_3D.shape[1] = sinogram_3D.shape[2]
            bins_shifted = np.random.randint(0, bins)
            angle = int(bins_shifted * 180/bins)

            image_2D = transforms.functional.rotate(image_2D, angle, fill=0) # Rotate image. Fill in unspecified pixels with zeros.
            sinogram_3D = torch.roll(sinogram_3D, bins_shifted, dims=(2,)) # Cycle (or 'Roll') sinogram by that angle
            sinogram_3D[:,:, 0:bins_shifted] = torch.flip(sinogram_3D[:,:,0:bins_shifted], dims=(1,)) # flip the cycled portion of the sinogram vertically

        return image_2D, sinogram_3D

    def VerticalFlip(image_2D, sinogram_3D):
        image_2D = torch.flip(image_2D,dims=(1,)) # Flip image vertically
        sinogram_3D = torch.flip(sinogram_3D,dims=(1,2)) # Flip sinogram horizontally and vertically
        return image_2D, sinogram_3D

    def HorizontalFlip(image_2D, sinogram_3D):
        image_2D = torch.flip(image_2D, dims=(2,)) # Flip image horizontally
        sinogram_3D = torch.flip(sinogram_3D, dims=(2,)) # Flip sinogram horizontally
        return image_2D, sinogram_3D

    ## Select Data ##
    image_2D = torch.from_numpy(image_array[index,:]) # image_2D.shape = (1, 71, 71)
    sinogram_3D = torch.from_numpy(sino_array[index,:]) # sinogram_3D.shape = (3,101,180,)

    ## Run Data Augmentation on Selected Data ##
    if augment==True:
        image_2D, sinogram_3D = RandRotate(image_2D, sinogram_3D)           # Always rotates image by a random angle
        if np.random.randn(1)[0]>0: # Half of the time, flips the image vertically
            image_2D, sinogram_3D = VerticalFlip(image_2D, sinogram_3D)
        if np.random.randn(1)[0]>0: # Half of the time, flips the image horizontally
            image_2D, sinogram_3D = HorizontalFlip(image_2D, sinogram_3D)

    ## Create A Set of Resized Outputs ##
    sinogram_3D_resize = transforms.Resize(size = (sino_size, sino_size), antialias=True)(sinogram_3D)
    image_2D_resize    = transforms.Resize(size = (image_size, image_size), antialias=True)(image_2D)

    ## Normalize Resized Outputs (optional) ##
    if SI_normalize:
        a = torch.reshape(image_2D_resize, (1,-1))
        a = nn.functional.normalize(a, p=1, dim = 1)
        image_2D_resize = torch.reshape(a, (1, image_size, image_size))
    if IS_normalize:
        a = torch.reshape(sinogram_3D_resize, (3,-1))                     # Flattens each sinogram.
        a = nn.functional.normalize(a, p=1, dim = 1)                      # Normalizes along dimension 1 (pixel values for each of the 3 channels)
        sinogram_3D_resize = torch.reshape(a, (3, sino_size, sino_size))  # Reshapes images back into square matrices.

    ## Adjust Output Channels of Resized Outputs ##
    if image_channels==1:
        image_out = image_2D_resize                 # For image_channels = 1, the image is just left alone
    else:
        image_out = image_2D_resize.repeat(3,1,1)   # For image_channels = 3, the image is repeated 3 times. This chould be altered to account for RGB images, etc.

    if sino_channels==1:
        sino_out = sinogram_3D_resize[0:1,:]        # Selects photopeak sinogram only
    else:
        sino_out = sinogram_3D_resize               # Keeps full sinogram with all channels

    # Returns both original and altered sinograms and images, assigned to CPU or GPU
    return sinogram_3D.to(device), IS_scale*sino_out.to(device), image_2D.to(device), SI_scale*image_out.to(device)

class NpArrayDataSet(Dataset):
    '''
    Class for loading data from .np files, given file directory strings and set of optional transformations.
    In the dataset used in our first two conference papers, the data repeat every 17500 steps but with different augmentations.
    For the dataset with FORE rebinning, the dataset contains no augmented examples. Augmentation is performed on the fly.
    '''
    def __init__(self, image_path, sino_path, config, image_size = 90, sino_size=90, image_channels=1, sino_channels=1,
                 augment=False, offset=0, num_examples=-1, sample_division=1):
        '''
        image_path:         path to images in data set
        sino_path:          path to sinograms in data set
        config:             configuration dictionary with hyperparameters
        image_size:         shape to resize image to (for output)
        image_channels:     number of channels in images
        sino_size:          shape to resize sinograms to (for output)
        sino_channels:      number of channels in sinograms (for photopeak sinograms, this is 1)
        augment:            Set True to perform on-the-fly augmentation of data set. Set False to not perform augmentation.
        offset:             To begin dataset at beginning of the datafile, set offset=0. To begin on the second image, offset = 1, etc.
        num_examples:       Max number of examples to load into dataset. Set to -1 to load the maximum number from the numpy array.
        sample_division:    set to 1 to use every example, 2 to use every other example, etc. (If sample_division=2, the dataset will be half the size.)
        '''
        ## Load Data to Arrays ##
        image_array = np.load(image_path, mmap_mode='r')       # self.image_tensor.shape=(#examples x1x71x71)
        sino_array = np.load(sino_path, mmap_mode='r')         # self.sinogram_tensor.shape=(#examples x3x101x180)

        ## Set Instance Variables ##
        if num_examples==-1:
            self.image_array = image_array[offset:,:]
            self.sino_array = sino_array[offset:,:]
        else:
            self.image_array = image_array[offset : offset + num_examples, :]
            self.sino_array = sino_array[offset : offset + num_examples, :]

        self.config = config
        self.image_size = image_size
        self.sino_size = sino_size
        self.image_channels = image_channels
        self.sino_channels = sino_channels
        self.augment = augment
        self.sample_division = sample_division

    def __len__(self):
        length = int(len(self.image_array)/sample_division)
        return length

    def __getitem__(self, idx):

        idx = idx*self.sample_division

        sino_ground, sino_ground_scaled, image_ground, image_ground_scaled = NpArrayDataLoader(self.image_array, self.sino_array, self.config, self.image_size,
                                                                                self.sino_size, self.image_channels, self.sino_channels,
                                                                                augment=self.augment, index=idx)

        return sino_ground, sino_ground_scaled, image_ground, image_ground_scaled
        # Returns both original, as well as altered, sinograms and images


# To Do

In [28]:
!ls -lh /content/drive/MyDrive/Colab/Working/dataset/sets/

total 83G
-rw------- 1 root root 709M Oct  8 02:41 test-actMap.npy
-rw------- 1 root root 709M Oct  8 02:41 test-anniMap.npy
-rw------- 1 root root 709M Oct  8 02:42 test-attenMap.npy
-rw------- 1 root root 709M Oct  8 02:42 test-highCountImage.npy
-rw------- 1 root root  13G Oct 12 22:15 test-highCountSino-382x513.npy
-rw------- 1 root root 709M Oct  8 02:46 test-obliqueImage.npy
-rw------- 1 root root 2.2G Oct  7 23:09 train-actMap.npy
-rw------- 1 root root 2.2G Oct  7 23:12 train-anniMap.npy
-rw------- 1 root root 2.2G Oct  7 23:13 train-attenMap.npy
-rw------- 1 root root 2.2G Oct  7 23:15 train-highCountImage.npy
-rw------- 1 root root 6.5G Oct 12 23:51 train-highCountSino-180x180.npy
-rw------- 1 root root  40G Oct 12 22:16 train-highCountSino-382x513.npy
-rw------- 1 root root 2.2G Oct  7 23:41 train-obliqueImage.npy
-rw------- 1 root root 452M Oct  8 01:46 val-actMap.npy
-rw------- 1 root root 452M Oct  8 01:46 val-anniMap.npy
-rw------- 1 root root 452M Oct  8 01:46 val-atten

Use os.path.join for paths instead of adding strings